A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef SimpleScheduleWTermConvention < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        startDate
        endDate
        tenor
        calendar
        convention
        termConvention
        endOfMonth
        dates
    end
    
    methods
        % constructor
        function schedule = SimpleScheduleWTermConvention(effectiveDate,terminationDate,tenor,calendar,convention,termConvention,dateGenRule,endOfMonth)
            
            schedule.tenor = tenor;
            schedule.calendar = calendar;
            schedule.convention = convention;
            schedule.termConvention = termConvention;
            schedule.endOfMonth = endOfMonth;
            
            dates= cell(2,1);
            
            dates{1} = effectiveDate;
            dates{2} = terminationDate;
            
            if strcmp(dateGenRule,'Backward')
                seed = terminationDate;
            else
                seed = effectiveDate;
            end
            
            numOfPeriod = 2;
            % adjust eom
            if endOfMonth && IsEndOfMonth(seed)
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
                end
            else
                if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
                    datesSize = length(dates);
                    for i=1:datesSize-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    % for CDS terminationDate is not adjusted
                else

%                     for i=1:numOfPeriod
%                         dates{i} =calendar.Adjust(dates{i},convention);
%                     end
                    
                    for i=1:numOfPeriod-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    if ~strcmp(termConvention,'Unadjusted')
                        dates{end} =calendar.Adjust(dates{end},termConvention);
                    end
                    
                end
            end
            
%             numStep = 3;
%             dtype = 'month';
%             dates = {};
%             % period handling
%             if strcmp(tenor(end),'D')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'day';
%             elseif strcmp(tenor(end),'M')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'month';
%             elseif strcmp(tenor(end),'Y')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'year';
%             else
%                 error('unimplemented')
%             end
%                       
%             if strcmp(dateGenRule,'Backward') || strcmp(dateGenRule,'BackwardCDS')
%                 temp_dates={};
%                 numOfPeriod = 0;
%                 i = 1;
%                 seed = terminationDate;
%                 while 1
%                     tempdate1 = AddDate(seed,-(i-1)*numStep,dtype);
%                     
%                     % when we add new date, if calendar & convention
%                     % adjusted dates is in the temp_dates then do not add
%                     % so that makes temp_dates unique !!
%                     if length(temp_dates) == 0
%                         temp_dates{end+1} = tempdate1;
%                         numOfPeriod = numOfPeriod + 1;
%                     else 
%                         if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
%                             temp_dates{end+1} = tempdate1;
%                             numOfPeriod = numOfPeriod + 1;
%                         end
%                     end
% %                     
% %                     temp_dates{end+1} = AddDate(seed,-(i-1)*numStep,dtype);
% %                     numOfPeriod = numOfPeriod + 1;
% 
%                     if temp_dates{numOfPeriod}.DateDiff(effectiveDate) <= 0 break; end
%                     i=i+1;
%                 end
%                 
%                 %% firstDate check
%                 %% if last generated Date is not equal to effectiveDate then we make it effectiveDate
%                 if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(effectiveDate,convention)
%                     temp_dates{end} = effectiveDate;
%                 end
%                 
%                 dates= cell(numOfPeriod,1);
%                 
%                 for i=1:numOfPeriod
%                     dates{i} = temp_dates{numOfPeriod-i+1};
%                 end
%                 
%             elseif strcmp(dateGenRule,'Forward') || strcmp(dateGenRule,'ForwardCDS')
%                 numOfPeriod = 0;
%                 i = 1;
%                 seed = effectiveDate;
%                 while 1
% 
%                     
%                     tempdate1 = AddDate(seed,(i-1)*numStep,dtype);
%                     if length(temp_dates) == 0
%                         temp_dates{end+1} = tempdate1;
%                         numOfPeriod = numOfPeriod + 1;
%                     else 
%                         if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
%                             temp_dates{end+1} = tempdate1;
%                             numOfPeriod = numOfPeriod + 1;
%                         end
%                     end
% %                     dates{end+1} = AddDate(seed,(i-1)*numStep,dtype);
% %                     numOfPeriod = numOfPeriod + 1;
%                     if dates{numOfPeriod}.DateDiff(terminationDate) >= 0 break; end
%                     i=i+1;
%                 end
%                 
%                 %% lastDate check
%                 %% if last generated Date is not equal to terminationDate then we make it terminationDate
%                 if calendar.Adjust(dates{end},convention) ~= calendar.Adjust(terminationDate,convention)
%                     dates{end} = terminationDate;
%                 end
%                 
%             elseif strcmp(dateGenRule,'TwentiethIMM')
%                 numOfPeriod = 0;
% %                 i = 1;
% %                 seed = effectiveDate;
%                 dates{end+1} = effectiveDate;
%                 seed = effectiveDate;
%                 numOfPeriod = numOfPeriod + 1;
%                 
%                 next20th = Next20th(effectiveDate);
%                 if DateDiff(effectiveDate,next20th) ~= 0
%                     dates{end+1} = next20th;
%                     seed = next20th;
%                     numOfPeriod = numOfPeriod + 1;
%                 end
%                 
%                 i = 1;
%                 while 1
%                     tempDate = AddDate(seed,(i)*numStep,dtype);
%                     
%                     if tempDate.DateDiff(terminationDate) < 0
%                         numOfPeriod = numOfPeriod + 1;
%                         dates{end+1} =tempDate;
%                         i=i+1;
%                     else
%                         break;
%                     end
% %                   
%                 end
%                 
%                 if DateDiff(dates{end},terminationDate) ~= 0
%                     dates{end+1} = Next20th(terminationDate);
%                 end
%                 
%             end
%             
%             % adjust eom
%             if endOfMonth && IsEndOfMonth(seed)
%                 for i=1:numOfPeriod
%                     dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
%                 end
%             else
%                 if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
%                     datesSize = length(dates);
%                     for i=1:datesSize-1
%                         dates{i} =calendar.Adjust(dates{i},convention);
%                     end
%                     
%                     % for CDS terminationDate is not adjusted
%                 else
% 
%                     for i=1:numOfPeriod
%                         dates{i} =calendar.Adjust(dates{i},convention);
%                     end
%                 end
%             end
%             
            
            schedule.dates = dates;
            
            schedule.startDate = dates{1};
            schedule.endDate = dates{end};
            
        end
        
        
        function out = print(schedule)
            out = cell(length(schedule.dates),1);
            for i=1:length(schedule.dates)
                out{i} = StrDate(schedule.dates{i},'str');
            end
        end
        
    end
    
end



In [ ]:
classdef SimpleSchedule < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        startDate
        endDate
        tenor
        calendar
        convention
        endOfMonth
        dates
    end
    
    methods
        % constructor
        function schedule = SimpleSchedule(effectiveDate,terminationDate,tenor,calendar,convention,dateGenRule,endOfMonth)
            
            schedule.tenor = tenor;
            schedule.calendar = calendar;
            schedule.convention = convention;
            schedule.endOfMonth = endOfMonth;
            
            dates= cell(2,1);
            
            dates{1} = effectiveDate;
            dates{2} = terminationDate;
            
            if strcmp(dateGenRule,'Backward')
                seed = terminationDate;
            else
                seed = effectiveDate;
            end
            
            numOfPeriod = 2;
            % adjust eom
            if endOfMonth && IsEndOfMonth(seed)
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
                end
            else
                if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
                    datesSize = length(dates);
                    for i=1:datesSize-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    % for CDS terminationDate is not adjusted
                else

                    for i=1:numOfPeriod
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                end
            end
            
%             numStep = 3;
%             dtype = 'month';
%             dates = {};
%             % period handling
%             if strcmp(tenor(end),'D')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'day';
%             elseif strcmp(tenor(end),'M')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'month';
%             elseif strcmp(tenor(end),'Y')
%                 numStep = str2double(tenor(1:end-1));
%                 dtype = 'year';
%             else
%                 error('unimplemented')
%             end
%                       
%             if strcmp(dateGenRule,'Backward') || strcmp(dateGenRule,'BackwardCDS')
%                 temp_dates={};
%                 numOfPeriod = 0;
%                 i = 1;
%                 seed = terminationDate;
%                 while 1
%                     tempdate1 = AddDate(seed,-(i-1)*numStep,dtype);
%                     
%                     % when we add new date, if calendar & convention
%                     % adjusted dates is in the temp_dates then do not add
%                     % so that makes temp_dates unique !!
%                     if length(temp_dates) == 0
%                         temp_dates{end+1} = tempdate1;
%                         numOfPeriod = numOfPeriod + 1;
%                     else 
%                         if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
%                             temp_dates{end+1} = tempdate1;
%                             numOfPeriod = numOfPeriod + 1;
%                         end
%                     end
% %                     
% %                     temp_dates{end+1} = AddDate(seed,-(i-1)*numStep,dtype);
% %                     numOfPeriod = numOfPeriod + 1;
% 
%                     if temp_dates{numOfPeriod}.DateDiff(effectiveDate) <= 0 break; end
%                     i=i+1;
%                 end
%                 
%                 %% firstDate check
%                 %% if last generated Date is not equal to effectiveDate then we make it effectiveDate
%                 if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(effectiveDate,convention)
%                     temp_dates{end} = effectiveDate;
%                 end
%                 
%                 dates= cell(numOfPeriod,1);
%                 
%                 for i=1:numOfPeriod
%                     dates{i} = temp_dates{numOfPeriod-i+1};
%                 end
%                 
%             elseif strcmp(dateGenRule,'Forward') || strcmp(dateGenRule,'ForwardCDS')
%                 numOfPeriod = 0;
%                 i = 1;
%                 seed = effectiveDate;
%                 while 1
% 
%                     
%                     tempdate1 = AddDate(seed,(i-1)*numStep,dtype);
%                     if length(temp_dates) == 0
%                         temp_dates{end+1} = tempdate1;
%                         numOfPeriod = numOfPeriod + 1;
%                     else 
%                         if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
%                             temp_dates{end+1} = tempdate1;
%                             numOfPeriod = numOfPeriod + 1;
%                         end
%                     end
% %                     dates{end+1} = AddDate(seed,(i-1)*numStep,dtype);
% %                     numOfPeriod = numOfPeriod + 1;
%                     if dates{numOfPeriod}.DateDiff(terminationDate) >= 0 break; end
%                     i=i+1;
%                 end
%                 
%                 %% lastDate check
%                 %% if last generated Date is not equal to terminationDate then we make it terminationDate
%                 if calendar.Adjust(dates{end},convention) ~= calendar.Adjust(terminationDate,convention)
%                     dates{end} = terminationDate;
%                 end
%                 
%             elseif strcmp(dateGenRule,'TwentiethIMM')
%                 numOfPeriod = 0;
% %                 i = 1;
% %                 seed = effectiveDate;
%                 dates{end+1} = effectiveDate;
%                 seed = effectiveDate;
%                 numOfPeriod = numOfPeriod + 1;
%                 
%                 next20th = Next20th(effectiveDate);
%                 if DateDiff(effectiveDate,next20th) ~= 0
%                     dates{end+1} = next20th;
%                     seed = next20th;
%                     numOfPeriod = numOfPeriod + 1;
%                 end
%                 
%                 i = 1;
%                 while 1
%                     tempDate = AddDate(seed,(i)*numStep,dtype);
%                     
%                     if tempDate.DateDiff(terminationDate) < 0
%                         numOfPeriod = numOfPeriod + 1;
%                         dates{end+1} =tempDate;
%                         i=i+1;
%                     else
%                         break;
%                     end
% %                   
%                 end
%                 
%                 if DateDiff(dates{end},terminationDate) ~= 0
%                     dates{end+1} = Next20th(terminationDate);
%                 end
%                 
%             end
%             
%             % adjust eom
%             if endOfMonth && IsEndOfMonth(seed)
%                 for i=1:numOfPeriod
%                     dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
%                 end
%             else
%                 if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
%                     datesSize = length(dates);
%                     for i=1:datesSize-1
%                         dates{i} =calendar.Adjust(dates{i},convention);
%                     end
%                     
%                     % for CDS terminationDate is not adjusted
%                 else
% 
%                     for i=1:numOfPeriod
%                         dates{i} =calendar.Adjust(dates{i},convention);
%                     end
%                 end
%             end
%             
            
            schedule.dates = dates;
            
            schedule.startDate = dates{1};
            schedule.endDate = dates{end};
            
        end
        
        
        function out = print(schedule)
            out = cell(length(schedule.dates),1);
            for i=1:length(schedule.dates)
                out{i} = StrDate(schedule.dates{i},'str');
            end
        end
        
    end
    
end



In [ ]:
classdef SimpleCashflow < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        amount
        paymentDate
    end
    
    methods
        % constructor
        function simpleCashflow = SimpleCashflow(amount,paymentDate)
            
            simpleCashflow.paymentDate = paymentDate;
            simpleCashflow.amount = amount;
                              
        end
        
        function out = Amount(simpleCashflow)
            out = simpleCashflow.amount;
        end
        
        function out = NPV(simpleCashflow,discountCurve)
            out = 0.0;
            
            t1 = DateDiff(simpleCashflow.paymentDate,discountCurve.params('asOfDate'))/365.0;
            if t1 > 0
                out = simpleCashflow.amount*discountCurve.DF(t1);
            else
                out = 0.0;
            end
        end
        
        % dummy BPS for simpleCashflow
        function out = BPS(simpleCashflow,discountCurve)
            out = 0.0;
        end
        
    end
        
end



In [ ]:
%siimul_NthToDefault_HazardRateBootStrapping_ytmDelta_New.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
curveShiftAmount = 0.0001;

simulNPV = zeros(100,3);
simulfairRate = zeros(100,3);

simulYTMList = zeros(1000,10);

deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
% deltaSimulationType = 'DiscPointByPointDelta';
% deltaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*4+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
else
    idxMax = 2*1+1;
end

numOfAsset = 3;


%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};
% 
dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% 우리은행 AAA
dataRate1 = {0.01926
0.02043
0.02129
0.02301
};

% % 신한카드 AA+
dataRate2 = {0.02039
0.02199
0.02302
0.02507
};

% 벤츠 파이넨셜 A+
dataRate3 = {0.02082
0.02394
0.02849
0.03151
};

% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataInOrigList = cell(3,1);

dataInList = {};
dataIn1.dataType = dataType;
dataIn1.dataTenor = dataTenor;
dataIn1.dataRate = dataRate1;
    
dataInList{1} = dataIn1;

dataInOrigList{1} = dataIn1;

dataIn2 = {};
dataIn2.dataType = dataType;
dataIn2.dataTenor = dataTenor;
dataIn2.dataRate = dataRate2;

dataInList{end+1} = dataIn2;

dataInOrigList{2} = dataIn2;

dataIn3 = {};
dataIn3.dataType = dataType;
dataIn3.dataTenor = dataTenor;
dataIn3.dataRate = dataRate3;

dataInList{end+1} = dataIn3;

dataInOrigList{3} = dataIn3;
% other static setting for bootstrapping

nthToDefaultSwapParams = {};
nthToDefaultSwapParams.recoveryRate = 0.4;


%% calendar
globalDict = containers.Map();
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(dateH,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;
 %% import and construct a discountCurve
%yldCurveSetting
yiledCurveSettings = containers.Map();

%KRW_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

curveName = 'KRW_KTB';
bootStrapFlag = true;
ycData = GetYieldCurveFromOctagon(dateH,curveName,globalDict,bootStrapFlag);


rawData = ycData.zcCurve.zeroCurve.params('rawData');
% rawData = curveOutDisc(:,1:2);
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end


% hazardRateCurveParams : bootStrapping input arg
hazardRateCurveParams = {};
hazardRateCurveParams.recoveryRate = 0.4;
survProbCurveList = cell(3,1);

for assetIter=1:numOfAsset
    
    for idxSimul =1:idxMax
    %initialize
    dataInList = dataInOrigList;
        % create marketDataSet for simulationTestCase
    dataRateOrig = dataInOrigList{assetIter}.dataRate;
    dataRateSim = dataRateOrig;
    
    if strcmp(deltaSimulationType,'ParallelDelta')   
    
%% YieldCurve simulation by parallel translation for delta

        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if idxSimul == 2

                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                end

            elseif idxSimul ==3
                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
    end
    
    dataInList{assetIter}.dataRate = dataRateSim;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,dateH,'KTB','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    nthToDefaultSwapParams.discountCurve=discountCurve;

    hazardRateCurveParams.discountCurve=discountCurve;
    % survProbCurveList
    %for riskyFixedBond 
    hazardRateCurveParams.settlesAccrual = false;
    hazardRateCurveParams.paysAtDefaultTime = false;

    hazardRateCurveParams.engineType = 'Integrate';
    % hazardRateCurveParams.engineType = 'Mid';

    %riskyFixedBond settings
    hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
    hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
    hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
    hazardRateCurveParams.riskyFixedBondPeriod = '3M';
    hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

    targetTimeBucket = [ 0.0027397 
    ];

    hazardRateCurveParams.targetTimeBucket = targetTimeBucket;


    for idx11=1:3

    curveOut = HazardRateCurveBootStrap('KRW',dateH,dataInList{idx11},hazardRateCurveParams);

    % survProbCurve
    rawDataSurv1 = zeros(length(curveOut.survProb),2);
    rawDataSurv1(:,1) = curveOut.times(:);
    rawDataSurv1(:,2) = curveOut.survProb(:);

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    {rawDataSurv1,dateH,'KRW','KRWSurvProb'});
    survProbCurveMktData = MktData(mktData);
    survProbCurveMktData.params('convention') = {};
    survProbCurveList{idx11} =     SurvProbCurve(survProbCurveMktData);

    end
    
    for idx11=1:3
        for idx22 = 1:length(dataInList{1}.dataRate)
            simulYTMList(3*idxMax*(assetIter-1) +idxMax*(idxSimul-1)+idx11,idx22) = dataInList{idx11}.dataRate{idx22};
        end
    end
    % % survProbCurve1
    % rawDataSurv1 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];


    % survProbCurve2
    % 
    % rawDataSurv2 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];
    % 
    % mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    % {rawDataSurv2,dateH,'KRW','KRWSurvProb'});
    % survProbCurveMktData = MktData(mktData);
    % survProbCurveMktData.params('convention') = {};
    % survProbCurve2 =  SurvProbCurve(survProbCurveMktData);
    % 
    % % survProbCurve3
    % 
    % rawDataSurv3 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];
    % 
    % mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    % {rawDataSurv3,dateH,'KRW','KRWSurvProb'});
    % survProbCurveMktData = MktData(mktData);
    % survProbCurveMktData.params('convention') = {};
    % survProbCurve3 =  SurvProbCurve(survProbCurveMktData);
    % 
    % survProbCurveList = {};
    % 
    % survProbCurveList{1} = survProbCurve1;
    % survProbCurveList{end+1} = survProbCurve2;
    % survProbCurveList{end+1} = survProbCurve3;

    nthToDefaultSwapParams.survProbCurveList = survProbCurveList;
    %nthToDefaultSwapParams settings
    nthToDefaultSwapParams.spreadCDSSettlementDays = 1;
    nthToDefaultSwapParams.spreadCDSBasis = 'Act365';
    nthToDefaultSwapParams.spreadCDSConvention = 'ModifiedFollowing';
    nthToDefaultSwapParams.spreadCDSPeriod = '3M';
    % nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
    % nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'BackwardCDS';
    nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'ForwardCDS';
    % 
    % nthToDefaultSwapParams.settlesAccrual = false;
    % nthToDefaultSwapParams.paysAtDefaultTime = false;

    nthToDefaultSwapParams.settlesAccrual = true;
    nthToDefaultSwapParams.paysAtDefaultTime = true;

    nthToDefaultSwapParams.engineType = 'Integrate';
    % nthToDefaultSwapParams.engineType = 'Mid';

    nthToDefaultSwapParams.evaluationDate = nthToDefaultSwapParams.discountCurve.asOfDate;

    nthToDefaultSwapParams.protectionStartDate = H_Date('2018-09-12');

    nthToDefaultSwapParams.startDate = nthToDefaultSwapParams.protectionStartDate;
    nthToDefaultSwapParams.endDate = AddTenor(nthToDefaultSwapParams.protectionStartDate,'5Y');
    jointCalendar = MakeCalendar(dateH,{},'30Y');
    nthToDefaultSwapParams.schedule = Schedule(nthToDefaultSwapParams.startDate,nthToDefaultSwapParams.endDate,nthToDefaultSwapParams.spreadCDSPeriod,jointCalendar, nthToDefaultSwapParams.spreadCDSConvention,...
                                  nthToDefaultSwapParams.spreadCDSDateGenerationRule,false);

    nthToDefaultSwapParams.protectionSideType = 'ProtectionBuyer';  
    nthToDefaultSwapParams.nominal = 10000.0;
    % nthToDefaultSwapParams.premiumRate = 0.010664814337304;
    nthToDefaultSwapParams.premiumRate = 0.018071323284456;


    % nthToDefaultSwap n = N, 1-> firstToDefault, 2->secondToDefault,...
    % nthToDefaultSwapParams.N = 1;
    nthToDefaultSwapParams.N = 1;

%     nthToDefaultParams.defaultCorrelation = 0.0 + 0.05*(idxSimul-1);   %0.85;
    nthToDefaultParams.defaultCorrelation = 0.90;   %0.85;
    
    nthToDefaultSwap = NthToDefaultSwap(nthToDefaultSwapParams.N,nthToDefaultSwapParams.protectionSideType,nthToDefaultSwapParams.engineType,nthToDefaultSwapParams.nominal,...
                                    nthToDefaultSwapParams.schedule,nthToDefaultSwapParams.premiumRate,nthToDefaultSwapParams.spreadCDSBasis,...
                                    nthToDefaultSwapParams.spreadCDSConvention,nthToDefaultSwapParams.settlesAccrual,nthToDefaultSwapParams.paysAtDefaultTime,...
                                    nthToDefaultSwapParams.protectionStartDate);


    outInfo = nthToDefaultSwap.Calculate(nthToDefaultSwapParams.recoveryRate,nthToDefaultParams.defaultCorrelation,nthToDefaultSwapParams.survProbCurveList,nthToDefaultSwapParams.discountCurve);

    simulNPV(idxSimul,assetIter) = outInfo.NPV;
    simulfairSpread(idxSimul,assetIter) = outInfo.fairSpread;


    end
end
aaa = 1.0;

aaa =1.0;


In [ ]:
%siimul_NthToDefault_HazardRateBootStrapping_ytmDelta_DiscDelta.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);

maxIdx = 10;
zcStep = 0.0001;
%shift size 10bp
curveShiftAmount = 0.0001;

simulNPV = zeros(100,3);
simulfairRate = zeros(100,3);

simulYTMList = zeros(1000,10);

% deltaSimulationType = 'ParallelDelta';
% deltaSimulationType = 'PointByPointDelta';

% deltaSimulationType = 'DiscParallelDelta';
deltaSimulationType = 'DiscPointByPointDelta';
% deltaSimulationType = 'None';

idxMax = 1;

if strcmp(deltaSimulationType,'ParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'PointByPointDelta')
    idxMax = 2*4+1;
elseif strcmp(deltaSimulationType,'DiscParallelDelta')
    idxMax = 2*1+1;
elseif strcmp(deltaSimulationType,'DiscPointByPointDelta')
    idxMax = 2*11+1;
else
    idxMax = 2*1+1;
end

numOfAsset = 3;


%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};
% 
dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% 우리은행 AAA
dataRate1 = {0.01926
0.02043
0.02129
0.02301
};

% % 신한카드 AA+
dataRate2 = {0.02039
0.02199
0.02302
0.02507
};

% 벤츠 파이넨셜 A+
dataRate3 = {0.02082
0.02394
0.02849
0.03151
};

% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataInOrigList = cell(3,1);

dataInList = {};
dataIn1.dataType = dataType;
dataIn1.dataTenor = dataTenor;
dataIn1.dataRate = dataRate1;
    
dataInList{1} = dataIn1;

dataInOrigList{1} = dataIn1;

dataIn2 = {};
dataIn2.dataType = dataType;
dataIn2.dataTenor = dataTenor;
dataIn2.dataRate = dataRate2;

dataInList{end+1} = dataIn2;

dataInOrigList{2} = dataIn2;

dataIn3 = {};
dataIn3.dataType = dataType;
dataIn3.dataTenor = dataTenor;
dataIn3.dataRate = dataRate3;

dataInList{end+1} = dataIn3;

dataInOrigList{3} = dataIn3;
% other static setting for bootstrapping

nthToDefaultSwapParams = {};
nthToDefaultSwapParams.recoveryRate = 0.4;

%% import and construct a discountCurve
curveName = 'KRW_KTB';
ycDiscData = GetYieldCurveFromOctagon(valueDate,curveName);
ycDiscDataOrig = ycDiscData;
yieldCurveParamsDisc = {};
 %KTB
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];

yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
% curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycDiscData,yieldCurveParamsDisc);
%     
% rawData = curveOutDisc(:,1:2);
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end


% hazardRateCurveParams : bootStrapping input arg
hazardRateCurveParams = {};
hazardRateCurveParams.recoveryRate = 0.4;
survProbCurveList = cell(3,1);

%KTB Delta
if strcmp(deltaSimulationType,'DiscParallelDelta') || strcmp(deltaSimulationType,'DiscPointByPointDelta')
    numOfAsset = 1;
end


for assetIter=1:numOfAsset
    
    for idxSimul =1:idxMax
        
    if strcmp(deltaSimulationType,'DiscParallelDelta')   
        dataRateOrig = ycDiscDataOrig.dataRate;
        dataRateSim = dataRateOrig;
    %% YieldCurve simulation by parallel translation for delta

        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if idxSimul == 2

                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                end

            elseif idxSimul ==3
                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
        ycDiscData.dataRate = dataRateSim;
        
    elseif   strcmp(deltaSimulationType,'DiscPointByPointDelta')
        dataRateOrig = ycDiscDataOrig.dataRate;
        dataRateSim = dataRateOrig;
        
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
        
        ycDiscData.dataRate = dataRateSim;
    end
    
    
    if strcmp(deltaSimulationType,'ParallelDelta')
        %initialize
        dataInList = dataInOrigList;
        % create marketDataSet for simulationTestCase
        dataRateOrig = dataInOrigList{assetIter}.dataRate;
        dataRateSim = dataRateOrig;
    
%% YieldCurve simulation by parallel translation for delta

        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if idxSimul == 2

                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                end

            elseif idxSimul ==3
                for idx2= 1: length(dataRateOrig)
                    dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
                end
            end
        end
        
         dataInList{assetIter}.dataRate = dataRateSim;
        
    elseif   strcmp(deltaSimulationType,'PointByPointDelta')
        %initialize
        dataInList = dataInOrigList;
        % create marketDataSet for simulationTestCase
        dataRateOrig = dataInOrigList{assetIter}.dataRate;
        dataRateSim = dataRateOrig;
        
        if idxSimul==1
            dataRateSim = dataRateOrig;
        else
            if mod(idxSimul,2) == 0
                idx2 = idxSimul/2;
                dataRateSim{idx2} = dataRateOrig{idx2} + curveShiftAmount*1.0;
                
            elseif mod(idxSimul,2) == 1
                idx2 = (idxSimul-1)/2;
                dataRateSim{idx2} = dataRateOrig{idx2} - curveShiftAmount*1.0;
            end
        end
        
         dataInList{assetIter}.dataRate = dataRateSim;
    end
    
    curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycDiscData,yieldCurveParamsDisc);
    
    rawData = curveOutDisc(:,1:2);

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,dateH,'KTB','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    nthToDefaultSwapParams.discountCurve=discountCurve;

    hazardRateCurveParams.discountCurve=discountCurve;
    % survProbCurveList
    %for riskyFixedBond 
    hazardRateCurveParams.settlesAccrual = false;
    hazardRateCurveParams.paysAtDefaultTime = false;

    hazardRateCurveParams.engineType = 'Integrate';
    % hazardRateCurveParams.engineType = 'Mid';

    %riskyFixedBond settings
    hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
    hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
    hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
    hazardRateCurveParams.riskyFixedBondPeriod = '3M';
    hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

    targetTimeBucket = [ 0.0027397 
    ];

    hazardRateCurveParams.targetTimeBucket = targetTimeBucket;


    for idx11=1:3

    curveOut = HazardRateCurveBootStrap('KRW',dateH,dataInList{idx11},hazardRateCurveParams);

    % survProbCurve
    rawDataSurv1 = zeros(length(curveOut.survProb),2);
    rawDataSurv1(:,1) = curveOut.times(:);
    rawDataSurv1(:,2) = curveOut.survProb(:);

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    {rawDataSurv1,dateH,'KRW','KRWSurvProb'});
    survProbCurveMktData = MktData(mktData);
    survProbCurveMktData.params('convention') = {};
    survProbCurveList{idx11} =     SurvProbCurve(survProbCurveMktData);

    end
    
    for idx11=1:3
        for idx22 = 1:length(dataInList{1}.dataRate)
            simulYTMList(3*idxMax*(assetIter-1) +idxMax*(idxSimul-1)+idx11,idx22) = dataInList{idx11}.dataRate{idx22};
        end
    end
    % % survProbCurve1
    % rawDataSurv1 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];


    % survProbCurve2
    % 
    % rawDataSurv2 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];
    % 
    % mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    % {rawDataSurv2,dateH,'KRW','KRWSurvProb'});
    % survProbCurveMktData = MktData(mktData);
    % survProbCurveMktData.params('convention') = {};
    % survProbCurve2 =  SurvProbCurve(survProbCurveMktData);
    % 
    % % survProbCurve3
    % 
    % rawDataSurv3 = [1.01369863	0.993992919
    % 2.010958904	0.98113753
    % 3.008219178	0.953832109
    % 5.005479452	0.913660385
    % ];
    % 
    % mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
    % {rawDataSurv3,dateH,'KRW','KRWSurvProb'});
    % survProbCurveMktData = MktData(mktData);
    % survProbCurveMktData.params('convention') = {};
    % survProbCurve3 =  SurvProbCurve(survProbCurveMktData);
    % 
    % survProbCurveList = {};
    % 
    % survProbCurveList{1} = survProbCurve1;
    % survProbCurveList{end+1} = survProbCurve2;
    % survProbCurveList{end+1} = survProbCurve3;

    nthToDefaultSwapParams.survProbCurveList = survProbCurveList;
    %nthToDefaultSwapParams settings
    nthToDefaultSwapParams.spreadCDSSettlementDays = 1;
    nthToDefaultSwapParams.spreadCDSBasis = 'Act365';
    nthToDefaultSwapParams.spreadCDSConvention = 'ModifiedFollowing';
    nthToDefaultSwapParams.spreadCDSPeriod = '3M';
    % nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
    % nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'BackwardCDS';
    nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'ForwardCDS';
    % 
    % nthToDefaultSwapParams.settlesAccrual = false;
    % nthToDefaultSwapParams.paysAtDefaultTime = false;

    nthToDefaultSwapParams.settlesAccrual = true;
    nthToDefaultSwapParams.paysAtDefaultTime = true;

    nthToDefaultSwapParams.engineType = 'Integrate';
    % nthToDefaultSwapParams.engineType = 'Mid';

    nthToDefaultSwapParams.evaluationDate = nthToDefaultSwapParams.discountCurve.asOfDate;

    nthToDefaultSwapParams.protectionStartDate = H_Date('2018-09-12');

    nthToDefaultSwapParams.startDate = nthToDefaultSwapParams.protectionStartDate;
    nthToDefaultSwapParams.endDate = AddTenor(nthToDefaultSwapParams.protectionStartDate,'5Y');
    jointCalendar = MakeCalendar(dateH,{},'30Y');
    nthToDefaultSwapParams.schedule = Schedule(nthToDefaultSwapParams.startDate,nthToDefaultSwapParams.endDate,nthToDefaultSwapParams.spreadCDSPeriod,jointCalendar, nthToDefaultSwapParams.spreadCDSConvention,...
                                  nthToDefaultSwapParams.spreadCDSDateGenerationRule,false);

    nthToDefaultSwapParams.protectionSideType = 'ProtectionBuyer';  
    nthToDefaultSwapParams.nominal = 10000.0;
    % nthToDefaultSwapParams.premiumRate = 0.010664814337304;
    nthToDefaultSwapParams.premiumRate = 0.018071323284456;


    % nthToDefaultSwap n = N, 1-> firstToDefault, 2->secondToDefault,...
    % nthToDefaultSwapParams.N = 1;
    nthToDefaultSwapParams.N = 1;

%     nthToDefaultParams.defaultCorrelation = 0.0 + 0.05*(idxSimul-1);   %0.85;
    nthToDefaultParams.defaultCorrelation = 0.90;   %0.85;
    
    nthToDefaultSwap = NthToDefaultSwap(nthToDefaultSwapParams.N,nthToDefaultSwapParams.protectionSideType,nthToDefaultSwapParams.engineType,nthToDefaultSwapParams.nominal,...
                                    nthToDefaultSwapParams.schedule,nthToDefaultSwapParams.premiumRate,nthToDefaultSwapParams.spreadCDSBasis,...
                                    nthToDefaultSwapParams.spreadCDSConvention,nthToDefaultSwapParams.settlesAccrual,nthToDefaultSwapParams.paysAtDefaultTime,...
                                    nthToDefaultSwapParams.protectionStartDate);


    outInfo = nthToDefaultSwap.Calculate(nthToDefaultSwapParams.recoveryRate,nthToDefaultParams.defaultCorrelation,nthToDefaultSwapParams.survProbCurveList,nthToDefaultSwapParams.discountCurve);

    simulNPV(idxSimul,assetIter) = outInfo.NPV;
    simulfairSpread(idxSimul,assetIter) = outInfo.fairSpread;


    end
end
aaa = 1.0;

aaa =1.0;


In [ ]:
%siimul_NthToDefault_HazardRateBootStrapping_ytmDelta.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);

simulNPV = zeros(1000,1);
simulfairSpread = zeros(1000,1);

%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};
% 
dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% 우리은행 AAA
dataRate1 = {0.01926
0.02043
0.02129
0.02301
};

% % 신한카드 AA+
dataRate2 = {0.02039
0.02199
0.02302
0.02507
};

% 벤츠 파이넨셜 A+
dataRate3 = {0.02082
0.02394
0.02849
0.03151
};

% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataInList = {};
dataIn1.dataType = dataType;
dataIn1.dataTenor = dataTenor;
dataIn1.dataRate = dataRate1;
    
dataInList{1} = dataIn1;

dataIn2 = {};
dataIn2.dataType = dataType;
dataIn2.dataTenor = dataTenor;
dataIn2.dataRate = dataRate2;

dataInList{end+1} = dataIn2;

dataIn3 = {};
dataIn3.dataType = dataType;
dataIn3.dataTenor = dataTenor;
dataIn3.dataRate = dataRate3;

dataInList{end+1} = dataIn3;

% other static setting for bootstrapping

nthToDefaultSwapParams = {};
nthToDefaultSwapParams.recoveryRate = 0.4;

 %% import and construct a discountCurve
curveName = 'KRW_KTB';
ycData = GetYieldCurveFromOctagon(valueDate,curveName);
yieldCurveParamsDisc = {};
 %KTB
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];

yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData,yieldCurveParamsDisc);
    
rawData = curveOutDisc(:,1:2);

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


% hazardRateCurveParams : bootStrapping input arg
hazardRateCurveParams = {};
hazardRateCurveParams.recoveryRate = 0.4;
survProbCurveList = cell(3,1);

for idxSimul =1:20


% dateH = H_Date(valueDate).AddDate(idxSimul-1,'day');   

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,dateH,'KTB','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);

nthToDefaultSwapParams.discountCurve=discountCurve;

hazardRateCurveParams.discountCurve=discountCurve;
% survProbCurveList
%for riskyFixedBond 
hazardRateCurveParams.settlesAccrual = false;
hazardRateCurveParams.paysAtDefaultTime = false;

hazardRateCurveParams.engineType = 'Integrate';
% hazardRateCurveParams.engineType = 'Mid';

%riskyFixedBond settings
hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
hazardRateCurveParams.riskyFixedBondPeriod = '3M';
hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

targetTimeBucket = [ 0.0027397 
];

hazardRateCurveParams.targetTimeBucket = targetTimeBucket;


for idx11=1:3
    
curveOut = HazardRateCurveBootStrap('KRW',dateH,dataInList{idx11},hazardRateCurveParams);

% survProbCurve
rawDataSurv1 = zeros(length(curveOut.survProb),2);
rawDataSurv1(:,1) = curveOut.times(:);
rawDataSurv1(:,2) = curveOut.survProb(:);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawDataSurv1,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurveList{idx11} =     SurvProbCurve(survProbCurveMktData);

end
% % survProbCurve1
% rawDataSurv1 = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];


% survProbCurve2
% 
% rawDataSurv2 = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
% {rawDataSurv2,dateH,'KRW','KRWSurvProb'});
% survProbCurveMktData = MktData(mktData);
% survProbCurveMktData.params('convention') = {};
% survProbCurve2 =  SurvProbCurve(survProbCurveMktData);
% 
% % survProbCurve3
% 
% rawDataSurv3 = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
% {rawDataSurv3,dateH,'KRW','KRWSurvProb'});
% survProbCurveMktData = MktData(mktData);
% survProbCurveMktData.params('convention') = {};
% survProbCurve3 =  SurvProbCurve(survProbCurveMktData);
% 
% survProbCurveList = {};
% 
% survProbCurveList{1} = survProbCurve1;
% survProbCurveList{end+1} = survProbCurve2;
% survProbCurveList{end+1} = survProbCurve3;

nthToDefaultSwapParams.survProbCurveList = survProbCurveList;
%nthToDefaultSwapParams settings
nthToDefaultSwapParams.spreadCDSSettlementDays = 1;
nthToDefaultSwapParams.spreadCDSBasis = 'Act365';
nthToDefaultSwapParams.spreadCDSConvention = 'ModifiedFollowing';
nthToDefaultSwapParams.spreadCDSPeriod = '3M';
% nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
% nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'BackwardCDS';
nthToDefaultSwapParams.spreadCDSDateGenerationRule = 'ForwardCDS';
% 
% nthToDefaultSwapParams.settlesAccrual = false;
% nthToDefaultSwapParams.paysAtDefaultTime = false;

nthToDefaultSwapParams.settlesAccrual = true;
nthToDefaultSwapParams.paysAtDefaultTime = true;

nthToDefaultSwapParams.engineType = 'Integrate';
% nthToDefaultSwapParams.engineType = 'Mid';

nthToDefaultSwapParams.evaluationDate = nthToDefaultSwapParams.discountCurve.asOfDate;

nthToDefaultSwapParams.protectionStartDate = H_Date('2018-09-12');

nthToDefaultSwapParams.startDate = nthToDefaultSwapParams.protectionStartDate;
nthToDefaultSwapParams.endDate = AddTenor(nthToDefaultSwapParams.protectionStartDate,'5Y');
jointCalendar = MakeCalendar(dateH,{},'30Y');
nthToDefaultSwapParams.schedule = Schedule(nthToDefaultSwapParams.startDate,nthToDefaultSwapParams.endDate,nthToDefaultSwapParams.spreadCDSPeriod,jointCalendar, nthToDefaultSwapParams.spreadCDSConvention,...
                              nthToDefaultSwapParams.spreadCDSDateGenerationRule,false);

nthToDefaultSwapParams.protectionSideType = 'ProtectionBuyer';  
nthToDefaultSwapParams.nominal = 10000.0;
% nthToDefaultSwapParams.premiumRate = 0.010664814337304;
nthToDefaultSwapParams.premiumRate = 0.018071323284456;


% nthToDefaultSwap n = N, 1-> firstToDefault, 2->secondToDefault,...
% nthToDefaultSwapParams.N = 1;
nthToDefaultSwapParams.N = 1;

nthToDefaultParams.defaultCorrelation = 0.0 + 0.05*(idxSimul-1);   %0.85;
nthToDefaultSwap = NthToDefaultSwap(nthToDefaultSwapParams.N,nthToDefaultSwapParams.protectionSideType,nthToDefaultSwapParams.engineType,nthToDefaultSwapParams.nominal,...
                                nthToDefaultSwapParams.schedule,nthToDefaultSwapParams.premiumRate,nthToDefaultSwapParams.spreadCDSBasis,...
                                nthToDefaultSwapParams.spreadCDSConvention,nthToDefaultSwapParams.settlesAccrual,nthToDefaultSwapParams.paysAtDefaultTime,...
                                nthToDefaultSwapParams.protectionStartDate);
                            

outInfo = nthToDefaultSwap.Calculate(nthToDefaultSwapParams.recoveryRate,nthToDefaultParams.defaultCorrelation,nthToDefaultSwapParams.survProbCurveList,nthToDefaultSwapParams.discountCurve);

simulNPV(idxSimul) = outInfo.NPV;
simulfairSpread(idxSimul) = outInfo.fairSpread;


end

aaa = 1.0;

aaa =1.0;


In [ ]:
function  [M,mv,alpha,unA] = ...
           select_taylor_degree(A,b,m_max,p_max,prec,shift,bal,force_estm)
%SELECT_TAYLOR_DEGREE   Select degree of Taylor approximation.
%   [M,MV,alpha,unA] = SELECT_TAYLOR_DEGREE(A,b,m_max,p_max) forms a matrix M
%   for use in determining the truncated Taylor series degree in EXPMV
%   and EXPMV_TSPAN, based on parameters m_max and p_max.
%   MV is the number of matrix-vector products with A or A^* computed.

%   Reference: A. H. Al-Mohy and N. J. Higham, Computing the action of
%   the matrix exponential, with an application to exponential
%   integrators. MIMS EPrint 2010.30, The University of Manchester, 2010.

%   Awad H. Al-Mohy and Nicholas J. Higham, November 9, 2010.

if nargin < 8, force_estm = false; end
if nargin < 4 || isempty(p_max), p_max = 8; end
if nargin < 3 || isempty(m_max), m_max = 55; end

if p_max < 2 || m_max > 60 || m_max + 1 < p_max*(p_max - 1)
    error('>>> Invalid p_max or m_max.')
end
n = length(A);
if nargin < 7 || isempty(bal), bal = false; end
if bal
    [D B] = balance(A);
    if norm(B,1) < norm(A,1), A = B; end
end
if nargin < 5 || isempty(prec), prec = class(A); end
switch prec
    case 'double'
        load theta_taylor
    case 'single'
        load theta_taylor_single
    case 'half'
        load theta_taylor_half
end
if shift
    mu = trace(A)/n;
    A = A-mu*speye(n);
end
mv = 0;
if ~force_estm, normA = norm(A,1); end

if ~force_estm && normA <= 4*theta(m_max)*p_max*(p_max + 3)/(m_max*size(b,2));
% if true
    % Base choice of m on normA, not the alpha_p.
    unA = 1;
    c = normA;
    alpha = c*ones(p_max-1,1);
else
    unA = 0;
    eta = zeros(p_max,1); alpha = zeros(p_max-1,1);
    for p = 1:p_max
        [c,k] = normAm(A,p+1);
        c = c^(1/(p+1));
        mv = mv + k;
        eta(p) = c;
    end
    for p = 1:p_max-1
        alpha(p) = max(eta(p),eta(p+1));
    end
end
M = zeros(m_max,p_max-1);
for p = 2:p_max
    for m = p*(p-1)-1 : m_max
        M(m,p-1) = alpha(p-1)/theta(m);
    end
end


In [ ]:
classdef Security < H_RootObject
    %Model is the basic object that represents a pricing model
 
    
    properties(SetAccess = public)
        payCcy
        startDate
        maturity
        endDate
        
    end
    
    methods
        function security = Security(params)
            if nargin > 0
                security.payCcy = params.payCcy;
                security.startDate = params.startDate;
                if isfield(params,'endDate')
                    security.endDate = params.endDate;
                    security.maturity = DateDiff(security.endDate,security.startDate)/365.0;
                
                else
                    % default
                    % maturity should be given in years
                    if isfield(params,'maturity')
                        security.maturity = params.maturity;
                    else
                        error('we need security maturity or endDate !!');
                    end
                    
                    security.endDate = security.startDate.AddDate(security.maturity,'year');
                end
                
            end
        end
        
  
    end
    
end



In [ ]:
function [ M ] = secondDivMatrix(xNodes)
%SECONDDIVMATRIX(xNodes) For a set of node points x_i, and a vector function
%evaluated at these points, f_i = f(x_i), form the matrix D, such that D*f = f'',
%where f'' is the second derivative estimate from central differencing at the node points. 
%see also firstDivMatrix
%
%Copyright (C) 2011 - present by OpenGamma Inc. and the OpenGamma group of companies
%Please see distribution for license.

dx = diff(xNodes);
N = length(xNodes);

a = 2./dx(1:end-1)./(dx(1:end-1)+dx(2:end));
b = -2./dx(1:end-1)./dx(2:end);
c = 2./dx(2:end)./(dx(1:end-1)+dx(2:end));

M = sparse(2:N-1,1:N-2,a,N,N) + sparse(2:N-1,2:N-1,b,N,N) + sparse(2:N-1,3:N,c,N,N);

% make it zero at the corner by default

% %forward diff
% M(1,1:3) = M(2,1:3);
% %backwards diff
% M(end,end-2:end)=M(end-1,end-2:end);

end



In [ ]:
classdef ScheduleWTermConvention < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        startDate
        endDate
        tenor
        calendar
        convention
        termConvention
        endOfMonth
        dates
    end
    
    methods
        % constructor
        function schedule = ScheduleWTermConvention(effectiveDate,terminationDate,tenor,calendar,convention,termConvention,dateGenRule,endOfMonth)
            
            schedule.tenor = tenor;
            schedule.calendar = calendar;
            schedule.convention = convention;
            schedule.termConvention = termConvention;
            % endOfMonth is only applied when tenor is longer than 1M
            schedule.endOfMonth = false;
            
            numStep = 3;
            dtype = 'month';
            dates = {};
            % period handling
            if strcmp(tenor(end),'D')
                numStep = str2double(tenor(1:end-1));
                dtype = 'day';
            elseif strcmp(tenor(end),'M')
                numStep = str2double(tenor(1:end-1));
                dtype = 'month';
                schedule.endOfMonth = true;
            elseif strcmp(tenor(end),'Y')
                numStep = str2double(tenor(1:end-1));
                dtype = 'year';
                schedule.endOfMonth= true;
            else
                error('unimplemented')
            end
                      
            if strcmp(dateGenRule,'Backward') || strcmp(dateGenRule,'BackwardCDS')
                temp_dates={};
                numOfPeriod = 0;
                i = 1;
                seed = terminationDate;
                while 1
                    tempdate1 = AddDate(seed,-(i-1)*numStep,dtype);
                    
                    % when we add new date, if calendar & convention
                    % adjusted dates is in the temp_dates then do not add
                    % so that makes temp_dates unique !!
                    if length(temp_dates) == 0
                        temp_dates{end+1} = tempdate1;
                        numOfPeriod = numOfPeriod + 1;
                    else 
                        if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
                            temp_dates{end+1} = tempdate1;
                            numOfPeriod = numOfPeriod + 1;
                        end
                    end
%                     
%                     temp_dates{end+1} = AddDate(seed,-(i-1)*numStep,dtype);
%                     numOfPeriod = numOfPeriod + 1;

                    if temp_dates{numOfPeriod}.DateDiff(effectiveDate) <= 0 break; end
                    i=i+1;
                end
                
                %% firstDate check
                %% if last generated Date is not equal to effectiveDate then we make it effectiveDate
                if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(effectiveDate,convention)
                    temp_dates{end} = effectiveDate;
                end
                
                dates= cell(numOfPeriod,1);
                
                for i=1:numOfPeriod
                    dates{i} = temp_dates{numOfPeriod-i+1};
                end
                
            elseif strcmp(dateGenRule,'Forward') || strcmp(dateGenRule,'ForwardCDS')
                numOfPeriod = 0;
                i = 1;
                seed = effectiveDate;
                while 1

                    
                    tempdate1 = AddDate(seed,(i-1)*numStep,dtype);
                    if length(temp_dates) == 0
                        temp_dates{end+1} = tempdate1;
                        numOfPeriod = numOfPeriod + 1;
                    else 
                        if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
                            temp_dates{end+1} = tempdate1;
                            numOfPeriod = numOfPeriod + 1;
                        end
                    end
%                     dates{end+1} = AddDate(seed,(i-1)*numStep,dtype);
%                     numOfPeriod = numOfPeriod + 1;
                    if dates{numOfPeriod}.DateDiff(terminationDate) >= 0 break; end
                    i=i+1;
                end
                
                %% lastDate check
                %% if last generated Date is not equal to terminationDate then we make it terminationDate
                if calendar.Adjust(dates{end},convention) ~= calendar.Adjust(terminationDate,convention)
                    dates{end} = terminationDate;
                end
                
            elseif strcmp(dateGenRule,'TwentiethIMM')
                numOfPeriod = 0;
%                 i = 1;
%                 seed = effectiveDate;
                dates{end+1} = effectiveDate;
                seed = effectiveDate;
                numOfPeriod = numOfPeriod + 1;
                
                next20th = Next20th(effectiveDate);
                if DateDiff(effectiveDate,next20th) ~= 0
                    dates{end+1} = next20th;
                    seed = next20th;
                    numOfPeriod = numOfPeriod + 1;
                end
                
                i = 1;
                while 1
                    tempDate = AddDate(seed,(i)*numStep,dtype);
                    
                    if tempDate.DateDiff(terminationDate) < 0
                        numOfPeriod = numOfPeriod + 1;
                        dates{end+1} =tempDate;
                        i=i+1;
                    else
                        break;
                    end
%                   
                end
                
                if DateDiff(dates{end},terminationDate) ~= 0
                    dates{end+1} = Next20th(terminationDate);
                end
                
            end
            
            % adjust eom
            if endOfMonth && IsEndOfMonth(seed)
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
                end
            else
                if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
                    datesSize = length(dates);
                    for i=1:datesSize-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    % for CDS terminationDate is not adjusted
                else

                    for i=1:numOfPeriod-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    if ~strcmp(termConvention,'Unadjusted')
                        dates{end} =calendar.Adjust(dates{end},termConvention);
                    end
                end
            end
            
            
            schedule.dates = dates;
            
            schedule.startDate = dates{1};
            schedule.endDate = dates{end};
            
        end
        
        
        function out = print(schedule)
            out = cell(length(schedule.dates),1);
            for i=1:length(schedule.dates)
                out{i} = StrDate(schedule.dates{i},'str');
            end
        end
        
    end
    
end



In [ ]:
classdef Schedule < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        startDate
        endDate
        tenor
        calendar
        convention
        endOfMonth
        dates
    end
    
    methods
        % constructor
        function schedule = Schedule(effectiveDate,terminationDate,tenor,calendar,convention,dateGenRule,endOfMonth)
            
            schedule.tenor = tenor;
            schedule.calendar = calendar;
            schedule.convention = convention;
            
            % endOfMonth is only applied when tenor is longer than 1M
            schedule.endOfMonth = false;
            
            numStep = 3;
            dtype = 'month';
            dates = {};
            % period handling
            if strcmp(tenor(end),'D')
                numStep = str2double(tenor(1:end-1));
                dtype = 'day';
            elseif strcmp(tenor(end),'M')
                numStep = str2double(tenor(1:end-1));
                dtype = 'month';
                schedule.endOfMonth = true;
            elseif strcmp(tenor(end),'Y')
                numStep = str2double(tenor(1:end-1));
                dtype = 'year';
                schedule.endOfMonth= true;
            else
                error('unimplemented')
            end
                      
            if strcmp(dateGenRule,'Backward') || strcmp(dateGenRule,'BackwardCDS')
                temp_dates={};
                numOfPeriod = 0;
                i = 1;
                seed = terminationDate;
                while 1
                    tempdate1 = AddDate(seed,-(i-1)*numStep,dtype);
                    
                    % when we add new date, if calendar & convention
                    % adjusted dates is in the temp_dates then do not add
                    % so that makes temp_dates unique !!
                    if length(temp_dates) == 0
                        temp_dates{end+1} = tempdate1;
                        numOfPeriod = numOfPeriod + 1;
                    else 
                        if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
                            temp_dates{end+1} = tempdate1;
                            numOfPeriod = numOfPeriod + 1;
                        end
                    end
%                     
%                     temp_dates{end+1} = AddDate(seed,-(i-1)*numStep,dtype);
%                     numOfPeriod = numOfPeriod + 1;

                    if temp_dates{numOfPeriod}.DateDiff(effectiveDate) <= 0 break; end
                    i=i+1;
                end
                
                %% firstDate check
                %% if last generated Date is not equal to effectiveDate then we make it effectiveDate
                if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(effectiveDate,convention)
                    temp_dates{end} = effectiveDate;
                end
                
                dates= cell(numOfPeriod,1);
                
                for i=1:numOfPeriod
                    dates{i} = temp_dates{numOfPeriod-i+1};
                end
                
            elseif strcmp(dateGenRule,'Forward') || strcmp(dateGenRule,'ForwardCDS')
                temp_dates={};
                numOfPeriod = 0;
                i = 1;
                seed = effectiveDate;
                while 1

                    
                    tempdate1 = AddDate(seed,(i-1)*numStep,dtype);
                    if length(temp_dates) == 0
                        temp_dates{end+1} = tempdate1;
                        numOfPeriod = numOfPeriod + 1;
                    else 
                        if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(tempdate1,convention)
                            temp_dates{end+1} = tempdate1;
                            numOfPeriod = numOfPeriod + 1;
                        end
                    end
%                     dates{end+1} = AddDate(seed,(i-1)*numStep,dtype);
%                     numOfPeriod = numOfPeriod + 1;
                    if temp_dates{numOfPeriod}.DateDiff(terminationDate) >= 0 break; end
%                     if dates{numOfPeriod}.DateDiff(terminationDate) >= 0 break; end
                    i=i+1;
                end
                
                %% lastDate check
                %% if last generated Date is not equal to terminationDate then we make it terminationDate
                if calendar.Adjust(temp_dates{end},convention) ~= calendar.Adjust(terminationDate,convention)
                    temp_dates{end} = terminationDate;
                end
                
%                 if calendar.Adjust(dates{end},convention) ~= calendar.Adjust(terminationDate,convention)
%                     dates{end} = terminationDate;
%                 end
                dates= cell(numOfPeriod,1);
                
                for i=1:numOfPeriod
                    dates{i} = temp_dates{i};
                end
                
            elseif strcmp(dateGenRule,'TwentiethIMM')
                numOfPeriod = 0;
%                 i = 1;
%                 seed = effectiveDate;
                dates{end+1} = effectiveDate;
                seed = effectiveDate;
                numOfPeriod = numOfPeriod + 1;
                
                next20th = Next20th(effectiveDate);
                if DateDiff(effectiveDate,next20th) ~= 0
                    dates{end+1} = next20th;
                    seed = next20th;
                    numOfPeriod = numOfPeriod + 1;
                end
                
                i = 1;
                while 1
                    tempDate = AddDate(seed,(i)*numStep,dtype);
                    
                    if tempDate.DateDiff(terminationDate) < 0
                        numOfPeriod = numOfPeriod + 1;
                        dates{end+1} =tempDate;
                        i=i+1;
                    else
                        break;
                    end
%                   
                end
                
                if DateDiff(dates{end},terminationDate) ~= 0
                    dates{end+1} = Next20th(terminationDate);
                end
                
            end
            
            % adjust eom
            if endOfMonth && IsEndOfMonth(seed)
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
                end
            else
                if strcmp(dateGenRule,'TwentiethIMM') || strcmp(dateGenRule,'BackwardCDS') || strcmp(dateGenRule,'ForwardCDS')
                    datesSize = length(dates);
                    for i=1:datesSize-1
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                    
                    % for CDS terminationDate is not adjusted
                else

                    for i=1:numOfPeriod
                        dates{i} =calendar.Adjust(dates{i},convention);
                    end
                end
            end
            
            
            schedule.dates = dates;
            
            schedule.startDate = dates{1};
            schedule.endDate = dates{end};
            
        end
        
        
        function out = print(schedule)
            out = cell(length(schedule.dates),1);
            for i=1:length(schedule.dates)
                out{i} = StrDate(schedule.dates{i},'str');
            end
        end
        
    end
    
end



In [ ]:
function SABRVol = SABRVolatility(strike,matT,lambda,alpha,beta,nu,SABRrho,forward)
%     strike = params.strike;
%     matT = params.matT;
%     lambda = params.lambda;
%     alpha = params.alpha;
%     beta = params.beta;
%     nu = params.nu;
%     SABRrho = params.SABRrho;
%     forward = params.forward;

    oneMinusBeta = 1.0 - beta;
    A = power(forward*strike,oneMinusBeta);
    sqrtA = sqrt(A);
    logM = log(forward/strike);
    z = (nu/alpha)*sqrtA*logM;
    B = 1.0-2.0*SABRrho*z +z*z;
    C = oneMinusBeta*oneMinusBeta*logM*logM;
    tmp = (sqrt(B)+z-SABRrho)/(1.0-SABRrho);
    xx = log(tmp);
    D= sqrtA*(1.0+C/24.0+C*C/1920.0);
    d = 1.0 + matT * (oneMinusBeta*oneMinusBeta*alpha*alpha/(24.0*A)+0.25*SABRrho*beta*nu*alpha/sqrtA+ ...
         (2.0-3.0*SABRrho*SABRrho)*(nu*nu/24.0));

    multiplier = 0.0;
    if xx<0.000001 && xx>-0.000001
        multiplier = 1.0;
    else
        multiplier = z/xx;
    end

    SABRVol = (alpha/D)*multiplier*d;
end



In [ ]:
function out = SABRVol(Forward,Strike,term,alpha,beta,rho,nu)
    alpha_adj = alpha;
    term_adj = term;
    nu_adj = nu;

    z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
    Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));

    multiplier = 0.0;
    if Xz < 0.000001 && Xz > -0.000001
        multiplier = 1.0;
    else
        multiplier = z/Xz;
    end

    model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
            * (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
                    + 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);


    out = model_vol;
          
end


In [ ]:
function out = SABRNormalVol(Forward,Strike,term,alpha,rho,nu)
    
%     z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
    z = nu/alpha * (Forward - Strike);
    Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));

    multiplier = 0.0;
    if Xz < 0.000001 && Xz > -0.000001
        multiplier = 1.0;
    else
        multiplier = z/Xz;
    end

    model_vol = multiplier * alpha * (1.0 + (2.0-3.0*power(rho, 2.0))/24.0*power(nu,2.0)*term);


    out = model_vol;
          
end


In [ ]:
classdef RoughHestonPade
   % Pade Approximation (rational function approximation) of
   % Rough Heston Vanilla option's closed form solution
    
    methods
        
        function out = hPade33(obj,params,a,x)
  
          H =params.H;
          rho = params.rho;
          nu = params.nu;
          al =H +.5;

          aa = sqrt(a.*(a + (0+1j)) - rho.^2 .* a.^2);
          rm = -(0+1j).* rho .* a - aa;
          rp = -(0+1j).* rho .* a + aa;

          b1 = -a.*(a+1j)./(2 .* gamma(1+al));
          b2 = (1-a.*1j) .* a.^2 .* rho./(2.* gamma(1+2.*al));               
          b3 = gamma(1+2*al)./gamma(1+3*al) .* ... 
            (a.^2.*(1j+a).^2./(8.*gamma(1+al).^2)+(a+1j).*a.^3.*rho.^2./(2.*gamma(1+2.*al)));

          g0 = rm;
          g1 = -rm./(aa.*gamma(1-al));
          g2 = rm./aa.^2/gamma(1-2.*al).*(1 + rm./(2.*aa).*gamma(1-2.*al)./gamma(1-al).^2);

          den = g0.^3 +2.*b1.*g0.*g1-b2.*g1.^2+b1.^2.*g2+b2.*g0.*g2;


          p1 = b1;
          p2 =(b1.^2.*g0.^2 + b2.*g0.^3 + b1.^3.*g1 + b1.*b2.*g0.*g1 - b2.^2.*g1.^2 +b1.*b3.*g1.^2 ...
              +b2.^2.*g0.*g2 - b1.*b3.*g0.*g2)./den;

          q1 = (b1.*g0.^2 + b1.^2.*g1 - b2.*g0.*g1 + b3.*g1.^2 - b1.*b2.*g2 -b3.*g0.*g2)./den;
          q2 = (b1.^2.*g0 + b2.*g0.^2 - b1.*b2.*g1 - b3.*g0.*g1 + b2.^2.*g2 - b1.*b3.*g2)./den;
          q3 = (b1.^3 + 2.*b1.*b2.*g0 + b3.*g0.^2 -b2.^2.*g1 +b1.*b3.*g1 )./den;
          p3 = g0.*q3;

          y = x^al;

          hPade = (p1.*y + p2.*y.^2 + p3.*y.^3)./(1 + q1.*y + q2.*y.^2 + q3.*y.^3);

          out = hPade;
            
        end
        
        function out = dhPade33(obj,params,a,x)
            H =params.H;
            rho = params.rho;
            nu = params.nu;
            alpha =H +0.5;
          
            aa  = sqrt(a.*(a+(0+1i)) - rho^2.*a.^2);
            rm  = -(0+1i)*rho.*a - aa;
            rp  = -(0+1i)*rho.*a + aa;

            b1  = -a .* (a+1i) / (2*gamma(1+alpha));
            b2  = (1-a*1i) .* a.^2 * rho / (2*gamma(1+2*alpha));  
            b3  = gamma(1+2*alpha) / gamma(1+3*alpha) * (a.^2.*(1i+a).^2 / (8*gamma(1+alpha)^2) + (a+1i).*a.^3*rho^2 / (2*gamma(1+2*alpha)));

            g0  = rm;
            g1  = -rm ./ (aa*gamma(1-alpha));
            g2  = rm ./ aa.^2 / gamma(1-2*alpha) .* (1 + rm./(2*aa)*gamma(1-2*alpha)/gamma(1-alpha)^2);

            den = g0.^3 + 2*b1.*g0.*g1 - b2.*g1.^2 + b1.^2.*g2 + b2.*g0.*g2;

            p1  = b1;
            p2  = (b1.^2.*g0.^2 + b2.*g0.^3 + b1.^3.*g1 + b1.*b2.*g0.*g1 - b2.^2.*g1.^2 + b1.*b3.*g1.^2 + b2.^2.*g0.*g2 - b1.*b3.*g0.*g2) ./ den;
            q1  = (b1.*g0.^2 + b1.^2.*g1 - b2.*g0.*g1 + b3.*g1.^2 - b1.*b2.*g2 - b3.*g0.*g2) ./ den;
            q2  = (b1.^2.*g0 + b2.*g0.^2 - b1.*b2.*g1 - b3.*g0.*g1 + b2.^2.*g2 - b1.*b3.*g2) ./ den;
            q3  = (b1.^3 + 2*b1.*b2.*g0 + b3.*g0.^2 - b2.^2.*g1 + b1.*b3.*g1) ./ den;
            p3  = g0 .* q3;
            
            y = x.^alpha;

          hPade = (p1.*y + p2.*y.^2 + p3.*y.^3)./(1 + q1.*y + q2.*y.^2 + q3.*y.^3);

          out = 1/2.*(hPade-rm).*(hPade-rp);
            
        end
        
        function out = dhPade33Old(obj,params,a,x)
  
          H =params.H;
          rho = params.rho;
          nu = params.nu;
          al =H +.5;

          aa = sqrt(a.*(a + (0+1j)) - rho.^2 .* a.^2);
          rm = -(0+1j).* rho .* a - aa;
          rp = -(0+1j).* rho .* a + aa;

          b1 = -a.*(a+1j)./(2 .* gamma(1+al));
          b2 = (1-a.*1j) .* a.^2 .* rho./(2.* gamma(1+2.*al));               
          b3 = gamma(1+2*al)./gamma(1+3*al) .* ... 
            (a.^2.*(1j+a).^2./(8.*gamma(1+al).^2)+(a+1j).*a.^3.*rho.^2./(2.*gamma(1+2.*al)));

          g0 = rm;
          g1 = -rm./(aa.*gamma(1-al));
          g2 = rm./aa.^2/gamma(1-2.*al).*(1 + rm./(2.*aa).*gamma(1-2.*al)./gamma(1-al).^2);

          den = g0.^3 +2.*b1.*g0.*g1-b2.*g1.^2+b1.^2.*g2+b2.*g0.*g2;


          p1 = b1;
          p2 =(b1.^2.*g0.^2 + b2.*g0.^3 + b1.^3.*g1 + b1.*b2.*g0.*g1 - b2.^2.*g1.^2 +b1.*b3.*g1.^2 ...
              +b2.^2.*g0.*g2 - b1.*b3.*g0.*g2)./den;

          q1 = (b1.*g0.^2 + b1.^2.*g1 - b2.*g0.*g1 + b3.*g1.^2 - b1.*b2.*g2 -b3.*g0.*g2)./den;
          q2 = (b1.^2.*g0 + b2.*g0.^2 - b1.*b2.*g1 - b3.*g0.*g1 + b2.^2.*g2 - b1.*b3.*g2)./den;
          q3 = (b1.^3 + 2.*b1.*b2.*g0 + b3.*g0.^2 -b2.^2.*g1 +b1.*b3.*g1 )./den;
          p3 = g0.*q3;

          y = x.^al;

          hPade = (p1.*y + p2.*y.^2 + p3.*y.^3)./(1 + q1.*y + q2.*y.^2 + q3.*y.^3);

%           out = hPade;
          out = 1/2.*(hPade-rm).*(hPade-rp);
            
        end
        
%         function out = dhPade33Fun(obj,params,u,t)
%             
%             dhPade33Fun1 = @(u,t) (obj.dhPade33(params,u,t));
%             out = dhPade33Fun1;
%         end
        
        function out = phiRoughHestonDhApproxRaw(obj,params,xiCurve,dhApprox,n,u,t)
  
            H  = params.H;
            rho= params.rho;
            nu = params.nu;
            al = H + 1/2;

            ti = (0:n)/n .* t;
            dt = t/n;
            x  = nu.^(1./al).*ti;
            xi = xiCurve(ti);
            dhApproxFun = @(a,t) (dhApprox(params,a,t));
            dah = dhApproxFun(u,x); 
            
%             out = exp(flip(xi)*dah'.*dt);
            out = exp(flip(xi)*dah.'*dt);

        end


                
        function out = phiRoughHestonDhApprox(obj,params,xiCurve,dhApprox,n,u,t)
            phi1 = @(a) (phiRoughHestonDhApproxRaw(obj,params,xiCurve,dhApprox,n,a,t));
            out = arrayfun(phi1, u);
        end
        
        
    end
end



In [ ]:
classdef RoughHestonAdams < Lewis
   % Fractional Ricatti Equation Solution by Adams scheme
   % The following code is a translation to R of Matlab code
   % written by Fabio Baschetti, Giacomo Bormetti, Pietro Rossi, 
   % and Silvia Romagnoli at the University of Bologna (2020).
   % back to matlab(2022)
   
    methods
        
        function out = hAGB(obj,H,rho,nu,n,a,bigT)
            
            DELTA = bigT/n;
            alpha = H + 1/2;

            hA = zeros(1,n+1);
            dhA = zeros(1,n+1);  
 
            F = @(u,h) (-0.5.*u.*(u+1i)+1i.*rho.*nu.*u.*h+0.5.*nu.^2.*h.^2);

            a01 = DELTA.^alpha./gamma(alpha+2).*alpha;
            a11 = DELTA.^alpha./gamma(alpha+2);
            b01 = DELTA.^alpha./gamma(alpha+1);
            dhA(1)= F(a,hA(1));
            % hA(1) is zero, boundary condition
            hP =  b01.*F(a,hA(1));
            hA(2) = a01.*F(a,hA(1)) + a11.*F(a,hP);
            dhA(2) = F(a,hA(2));
            
            % ------

            k =  1:(n-1);
            a0 = DELTA.^alpha./gamma(alpha+2).*(k.^(alpha+1) - (k-alpha).*(k+1).^alpha);
            q  = 0:(n-1);
            aj = DELTA.^alpha./gamma(alpha+2).*((q+2).^(alpha+1) + (q).^(alpha+1) - 2.*(q+1).^(alpha+1));
            r  = 0:n;
            bj = DELTA.^alpha./gamma(alpha+1).*((r+1).^alpha - (r).^alpha);
            akp1 =DELTA.^alpha./gamma(alpha+2);

            for kk =1:(n-1)
                f = F(a,hA(1:(kk+1)));
                aA = [a0(kk), flip(aj(1:kk))];
                hP = transpose(f) * flip(bj(1:(kk+1)));
                hA(kk+2) = transpose(f) * aA + akp1*F(a,hP);
                dhA(k+2) = F(a,hA(kk+2));
            end

            out = hA;
            
        end
        
        function out = dhAGB(obj,H,rho,nu,n,a,bigT)
            
            DELTA = bigT/n;
            alpha = H + 1/2;

            hA = zeros(1,n+1);
            dhA = zeros(1,n+1);  
 
            F = @(u,h) (-0.5.*u.*(u+1i)+1i.*rho.*nu.*u.*h+0.5.*nu.^2.*h.^2);

            a01 = DELTA.^alpha./gamma(alpha+2).*alpha;
            a11 = DELTA.^alpha./gamma(alpha+2);
            b01 = DELTA.^alpha./gamma(alpha+1);
            dhA(1)= F(a,hA(1));
            % hA(1) is zero, boundary condition
            hP =  b01.*F(a,hA(1));
            hA(2) = a01.*F(a,hA(1)) + a11.*F(a,hP);
            dhA(2) = F(a,hA(2));
            
            % ------

            k =  1:(n-1);
            a0 = DELTA.^alpha./gamma(alpha+2).*(k.^(alpha+1) - (k-alpha).*(k+1).^alpha);
            q  = 0:(n-1);
            aj = DELTA.^alpha./gamma(alpha+2).*((q+2).^(alpha+1) + (q).^(alpha+1) - 2.*(q+1).^(alpha+1));
            r  = 0:n;
            bj = DELTA.^alpha./gamma(alpha+1).*((r+1).^alpha - (r).^alpha);
            akp1 =DELTA.^alpha./gamma(alpha+2);

            for kk =1:(n-1)
                f = F(a,hA(1:(kk+1)));
                aA = [a0(kk), flip(aj(1:kk))];
                
%                 hP = transpose(f) * flip(bj(1:(kk+1)));
%                 hA(kk+2) = transpose(f) * aA + akp1*F(a,hP);
                
                hP = sum(f.* flip(bj(1:(kk+1))));
                hA(kk+2) = sum(f.*aA) + akp1*F(a,hP);
                
                dhA(kk+2) = F(a,hA(kk+2));
            end

            out = dhA;
            
        end
        
%         function out = dhAGB(obj,H,rho,nu,n,a,bigT)
%              out = arrayfun(@(a) obj.dhAGBRaw(H,rho,nu,n,a,bigT),a);
%                                      
%         end
        
        function out = phiRoughHestonRaw(obj,params,xiCurve,n,u,t)
  
            H  = params.H;
            rho= params.rho;
            nu = params.nu;
            al = H + 1/2;

            ti = (0:n)/n .* t;
            dt = t/n;
%             x  = nu.^(1./al).*ti;
            xi = xiCurve(ti);
            dah = obj.dhAGB(H,rho,nu,n,u,t);
            
            out = exp(flip(xi)*dah.'*dt);

        end


        function out = phiRoughHeston(obj,params,xiCurve,n,u,t)
            out = arrayfun(@(u) obj.phiRoughHestonRaw(params,xiCurve,n,u,t),u);
        end
                
        function out = yAlpha(obj,H,tau,xiCurve)
            alpha = H + 0.5;
            integ = @(s) (xiCurve(s)./(tau-s).^alpha);
            res = integral(integ,0.0,tau)/gamma(1.0-alpha);
            out = res;
        end
        
        function out = FindaAmax(obj,params,xiCurve,k,tau,epsSig)
            rho = params.rho;
            H = params.H;
            nu = params.nu;
            
            yy = sqrt(1-rho*rho)/nu *obj.yAlpha(H,tau,xiCurve);
            c = sqrt(pi/2)*exp(-k/2)*epsSig;
            f = @(a) (exp(-yy*a) -c*yy*(a*a+0.25));
            options = optimset('TolX',1e-4);
            amax = fzero(f,[0 1.0/sqrt(c*yy)],options);
            out = amax;
        end
        
        % Now we compute rough Heston option prices
        function out = otmRoughHestonRaw(obj, params, xiCurve, nSteps,k,t) 

            k_minus = (k<0).*k;
            
            phi = @(u,t) (obj.phiRoughHeston(params, xiCurve, nSteps,u,t));
            epsSig = 1e-4;
            aMax = obj.FindaAmax(params, xiCurve, k, t, epsSig);
            integrand = @(u) (real(exp(-1j.*u.*k).*phi(u-1j./2,t)./(u.^2+ 1/4)));
            out = exp(k_minus) - exp(k./2)./pi .* integral(integrand,0,aMax,'RelTol',1e-8);
      
        end
        
        function out = otmRoughHeston(obj, params, xiCurve, nSteps,k,t) 
%             phi1 = @(k) (obj.otmRoughHestonRaw(params,xiCurve,n,k,t));
            out = arrayfun(@(k) obj.otmRoughHestonRaw(params,xiCurve,nSteps,k,t),k);
        end
        
        function out = impliedVolRoughHeston(obj,params, xiCurve, nSteps)
%             phi1 = obj.otmRoughHeston(params, xiCurve, nSteps);
            out = @(k,t) (obj.ivOTM(1,exp(k),t,obj.otmRoughHeston(params, xiCurve, nSteps,k,t)));
            
        end
        
    end
end



In [ ]:
function [ output ] = rotation33(theta1,theta2,angle)
% 3D rotation matrix
% theta1, theat2 is for a rotation axis in 3D
% angle is the rotation angle

    rotation = makehgtform('axisrotate',[cos(theta1)*sin(theta2),cos(theta1)*cos(theta2),sin(theta1)],angle);
    
    output = rotation(1:3,1:3);
end



In [ ]:
function [ output ] = rotation22(theta)
    rotation = zeros(2,2);
    rotation(1,1) =  cos(theta);
    rotation(1,2) = -sin(theta);
    rotation(2,1) =  sin(theta);
    rotation(2,2) =  cos(theta);
    
    output = rotation;
end



In [ ]:
classdef RiskyFixedBondHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        
        rateType
        
        rate
        rateTenor
        
        riskyFixedBondSettlementDays
        fixingCalendar
        riskyFixedBondBasis
        riskyFixedBondConvention
        riskyFixedBondPeriod
        riskyFixedBondDateGenerationRule
        recoveryRate
        discountCurve
        
        evaluationDate
        
        settlesAccrual
        paysAtDefaultTime
        engineType
        
        riskyFixedBond
        schedule
        survProbCurve
%         discountCurve
%         protectionStartDate
        efffectiveStartDate
        
        earliestDate
        latestDate
        
    end
    
    methods
        % constructor
        function riskyFixedBondHelper = RiskyFixedBondHelper(params)
                if nargin > 0
                    riskyFixedBondHelper.rateType = 'RiskyFixedBond';
                    
                    riskyFixedBondHelper.rate = params('rate');
                    riskyFixedBondHelper.rateTenor = params('rateTenor');
                    
                    riskyFixedBondHelper.riskyFixedBondSettlementDays = params('riskyFixedBondSettlementDays');
                    riskyFixedBondHelper.fixingCalendar = params('fixingCalendar');
                    riskyFixedBondHelper.riskyFixedBondBasis = params('riskyFixedBondBasis');
                    riskyFixedBondHelper.riskyFixedBondConvention = params('riskyFixedBondConvention');
                    riskyFixedBondHelper.riskyFixedBondPeriod = params('riskyFixedBondPeriod');
                    riskyFixedBondHelper.riskyFixedBondDateGenerationRule = params('riskyFixedBondDateGenerationRule');
                    riskyFixedBondHelper.recoveryRate  = params('recoveryRate');
                    riskyFixedBondHelper.discountCurve = params('discountCurve');
                    
                    riskyFixedBondHelper.evaluationDate = riskyFixedBondHelper.discountCurve.asOfDate;
                    
                    if ~isKey(params,'settlesAccrual')
                        riskyFixedBondHelper.settlesAccrual = true;
                    else
                        riskyFixedBondHelper.settlesAccrual = params('settlesAccrual');
                    end
                    
                    if ~isKey(params,'paysAtDefaultTime')
                        riskyFixedBondHelper.paysAtDefaultTime = true;
                    else
                        riskyFixedBondHelper.paysAtDefaultTime = params('paysAtDefaultTime');
                    end
                    
                    if ~isKey(params,'engineType')
                        riskyFixedBondHelper.engineType = engineType;
                    else
                        riskyFixedBondHelper.engineType = params('engineType');
                    end
                    
                    riskyFixedBondHelper.riskyFixedBond = MakeRiskyFixedBond(riskyFixedBondHelper);
                    riskyFixedBondHelper.survProbCurve = SurvProbCurve();
                    riskyFixedBondHelper.survProbCurve.params('asOfDate') = riskyFixedBondHelper.discountCurve.asOfDate;
                else
                    riskyFixedBondHelper.rate = 0.02;
%                     riskyFixedBondHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(riskyFixedBondHelper);
                
                riskyFixedBondHelper.earliestDate = out.earliestDate;
                riskyFixedBondHelper.latestDate = out.latestDate;
                
        end
        
        function out = MakeRiskyFixedBond(riskyFixedBondHelper)
            referenceDate = riskyFixedBondHelper.evaluationDate;
            
            % for CDS start date is not adjusted
            riskyFixedBondHelper.efffectiveStartDate = referenceDate.AddDate(riskyFixedBondHelper.riskyFixedBondSettlementDays,'day');
%            startDate = spreadCDSHelper.fixingCalendar.Adjust(protectionStartDate,spreadCDSHelper.spreadCDSConvention);
            
            startDate = riskyFixedBondHelper.efffectiveStartDate;
            endDate = AddTenor(startDate,riskyFixedBondHelper.rateTenor);
            % fixed & floating schedule 
            fixedSchedule = Schedule(startDate,endDate,riskyFixedBondHelper.riskyFixedBondPeriod ,riskyFixedBondHelper.fixingCalendar,...
                                    riskyFixedBondHelper.riskyFixedBondConvention, riskyFixedBondHelper.riskyFixedBondDateGenerationRule,false); 
                                
            riskyFixedBondHelper.schedule = fixedSchedule;
             
            type = 'Buyer';
            engineType = riskyFixedBondHelper.engineType;
            
            nominal = 1.0;
%             spread = 0.01;
            rate = riskyFixedBondHelper.rate;
            
            
            riskyFixedBond = RiskyFixedBond(type,engineType,nominal,fixedSchedule,rate,riskyFixedBondHelper.riskyFixedBondBasis, ...
                                     riskyFixedBondHelper.riskyFixedBondConvention,riskyFixedBondHelper.settlesAccrual,...
                                     riskyFixedBondHelper.paysAtDefaultTime,riskyFixedBondHelper.efffectiveStartDate);
                                  
            out = riskyFixedBond;
        end
        
        function out = MakeSchedule(riskyFixedBondHelper)
            referenceDate = riskyFixedBondHelper.evaluationDate;
            riskyFixedBondHelper.efffectiveStartDate = referenceDate.AddDate(riskyFixedBondHelper.riskyFixedBondSettlementDays,'day');
            
%             startDate = spreadCDSHelper.fixingCalendar.Adjust(protectionStartDate,spreadCDSHelper.spreadCDSConvention);
            % for CDS start date is not adjusted
            
            startDate = riskyFixedBondHelper.efffectiveStartDate;
            endDate = AddTenor(startDate,riskyFixedBondHelper.rateTenor);
            % fixed & floating schedule 
            schedule = Schedule(startDate,endDate,riskyFixedBondHelper.riskyFixedBondPeriod ,riskyFixedBondHelper.fixingCalendar,...
                                    riskyFixedBondHelper.riskyFixedBondConvention, riskyFixedBondHelper.riskyFixedBondDateGenerationRule,false); 
            
                                  
             out = schedule;
        end
        
        function out = InitializeDates(riskyFixedBondHelper)
            
            %earliestDate
            earliestDate = riskyFixedBondHelper.schedule.dates{1};
            latestDate = riskyFixedBondHelper.fixingCalendar.Adjust(riskyFixedBondHelper.schedule.dates{end},riskyFixedBondHelper.riskyFixedBondConvention);
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
           
        end
        
        function out = ImpliedQuote(riskyFixedBondHelper)
            outInfo = riskyFixedBondHelper.riskyFixedBond.Calculate(riskyFixedBondHelper.recoveryRate,...
                                      riskyFixedBondHelper.survProbCurve,riskyFixedBondHelper.discountCurve);
%             out = outInfo.fairRate;
            out = outInfo.NPV;
            
        end
        
        % bootstrapping
        function out = QuoteError(riskyFixedBondHelper)
%             out = riskyFixedBondHelper.rate - riskyFixedBondHelper.ImpliedQuote();
            
        out = 1.0 - riskyFixedBondHelper.ImpliedQuote();
            
        end
        
        function out = BootStrapError(riskyFixedBondHelper,guess,idx)
            riskyFixedBondHelper.Update(idx,guess);
            out = riskyFixedBondHelper.QuoteError();
        end
        
        function Update(riskyFixedBondHelper,idx,guess)
             % forwardCurve update
            forwardRawData = riskyFixedBondHelper.survProbCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            riskyFixedBondHelper.survProbCurve.params('rawData')= forwardRawData;
            
        end
        
        
    end
    
end



In [ ]:
classdef RiskyFixedBond < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        type
        engineType
        nominal
        fixedSchedule
        rate
        fixedDayCounter
        fixedConvention
        
        settlesAccrual
        paysAtDefaultTime
        
        efffectiveStartDate
        
        fixedRateLeg
%         upfrontPayment
        
        NPV;
%         fairRate;
       
    end
    
    methods
        % constructor
        function riskyFixedBond = RiskyFixedBond(type,engineType,nominal,fixedSchedule,rate,fixedDayCounter, ...
                                      fixedConvention,settlesAccrual,paysAtDefaultTime,efffectiveStartDate)
            
            riskyFixedBond.type = type;
            riskyFixedBond.engineType = engineType;
            
            riskyFixedBond.nominal = nominal;
            riskyFixedBond.fixedSchedule = fixedSchedule;
            riskyFixedBond.rate = rate;
            riskyFixedBond.fixedDayCounter = fixedDayCounter;
            riskyFixedBond.fixedConvention = fixedConvention;
            
            riskyFixedBond.settlesAccrual = settlesAccrual;
            riskyFixedBond.paysAtDefaultTime = paysAtDefaultTime;
            
            riskyFixedBond.efffectiveStartDate = efffectiveStartDate;
            
            % bug fixed 20180913
            % when yield curve building, fixedLeg's payment date convention
            % is given as floatingLeg's payment date convention in quantlib
            % !!!!
            
%             vanillaSwap.fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedSchedule.convention);
%             riskyFixedBond.fixedRateLeg = FixedRateLeg(fixedSchedule,nominal,rate,fixedDayCounter,fixedConvention);
            riskyFixedBond.fixedRateLeg = FixedRateLeg(fixedSchedule,nominal,rate,fixedDayCounter,fixedConvention,0);
            
            % notional redemption at maturity
            riskyFixedBond.fixedRateLeg.couponRates{end+1} = SimpleCashflow(1.0,fixedSchedule.dates{end});
            
            
            
        end
        
        function out = StartDate(riskyFixedBond)
            d1 = riskyFixedBond.fixedRateLeg.StartDate();
            out = d1;
%             d2 = vanillaSwap.floatingLeg.StartDate();
            
%             if DateDiff(d1,d2) > 0
%                 out = d2;
%             else
%                 out = d1;
%             end

        end
        
        function out = EndDate(riskyFixedBond)
            d1 = riskyFixedBond.fixedRateLeg.EndDate();
            out = d1;
%             d2 = vanillaSwap.floatingLeg.EndDate();
            
%             if DateDiff(d1,d2) > 0
%                 out = d1;
%             else
%                 out = d2;
%             end

        end
        
        function out = Calculate(riskyFixedBond,recoveryRate,survProbCurve,discountCurve)
            if strcmp(riskyFixedBond.engineType,'Integrate')

                out = Calculate_Integral(riskyFixedBond,recoveryRate,survProbCurve,discountCurve);
            else
                out = Calculate_Mid(riskyFixedBond,recoveryRate,survProbCurve,discountCurve);
            end
        
        end
        
        function out = Calculate_Mid(riskyFixedBond,recoveryRate,survProbCurve,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurve.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            redemptionNPV = 0.0;
            
            legSize = length(riskyFixedBond.fixedRateLeg.couponRates);
            
            startDate = riskyFixedBond.efffectiveStartDate;
            for i=1:legSize
                fixedRateCoupon = riskyFixedBond.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                endDate = paymentDate;
                
                
                if DateDiff(startDate,evaluationDate) <= 0
                    startDate = evaluationDate;
                end
                
                defaultDate = startDate.AddDate(floor(DateDiff(endDate,startDate)/2),'day');
                
                S = survProbCurve.SurvProbFromDate(paymentDate);
                P = survProbCurve.DefaultProbFromDate(startDate,endDate);
                
                if i < legSize                              
                    couponLegNPV = couponLegNPV + S * riskyFixedBond.nominal * 1.0 * fixedRateCoupon.Amount() * ...
                                                      discountCurve.DFFromDate(paymentDate);
                else
                    redemptionNPV = redemptionNPV + S * riskyFixedBond.nominal * 1.0 * fixedRateCoupon.Amount() * ...
                                                      discountCurve.DFFromDate(paymentDate);
                end
                % accrual cashflow in case of default
                % No accrual cashflow in case of default for riskyFixedBond
                
                % at default cashflow
                % default cashflow happen at defaultTime for
                % riskyFixedBond!!
                
                claim = riskyFixedBond.nominal * (recoveryRate);
                defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(defaultDate);
                
                %loop iteration
                startDate =  endDate;
            end

            
            out.NPV = defaultLegNPV + couponLegNPV + redemptionNPV;
            out.fairRate = (riskyFixedBond.nominal*1.0 - defaultLegNPV - redemptionNPV)*riskyFixedBond.rate/couponLegNPV;
            
        end
        
        function out = Calculate_Integral(riskyFixedBond,recoveryRate,survProbCurve,discountCurve)
            % MidPointCDS Engine !
            
            % consistency check !
            if DateDiff(survProbCurve.asOfDate,discountCurve.asOfDate) ~= 0
                error('inconsistency ! survProbCurve and discountCurve asOfDate should be the same !')
            end
            
            evaluationDate = discountCurve.asOfDate;
            
            couponLegNPV = 0.0;
            defaultLegNPV = 0.0;
            redemptionNPV = 0.0;
            
            legSize = length(riskyFixedBond.fixedRateLeg.couponRates);
            
            startDate = riskyFixedBond.efffectiveStartDate;
            for i=1:legSize
                fixedRateCoupon = riskyFixedBond.fixedRateLeg.couponRates{i};
                if DateDiff(fixedRateCoupon.paymentDate,evaluationDate) <= 0
                    continue;
                end
                
                paymentDate = fixedRateCoupon.paymentDate;
                endDate = paymentDate;
                
                
                if DateDiff(startDate,evaluationDate) <= 0
                    startDate = evaluationDate;
                end
                
%                 defaultDate = startDate.AddDate(floor(DateDiff(endDate,startDate)/2),'day');
                
                S = survProbCurve.SurvProbFromDate(paymentDate);
%                 P = survProbCurve.DefaultProbFromDate(startDate,endDate);
                
                if i < legSize                              
                    couponLegNPV = couponLegNPV + S * riskyFixedBond.nominal * 1.0 * fixedRateCoupon.Amount() * ...
                                                      discountCurve.DFFromDate(paymentDate);
                else
                    redemptionNPV = redemptionNPV + S * riskyFixedBond.nominal * 1.0 * fixedRateCoupon.Amount() * ...
                                                      discountCurve.DFFromDate(paymentDate);
                end
                % accrual cashflow in case of default
                % No accrual cashflow in case of default for riskyFixedBond
                
                % at default cashflow
                % default cashflow happen at defaultTime for
                % riskyFixedBond!!
                
                date0 = startDate;
                stepSize = 7;
                date1 = date0.AddDate(stepSize,'day');
                
                %date1 should be earlier than endDate
                % forward generation of the schedule is dangerous!!
                % check for the theta in the future
                
                if DateDiff(date1,endDate) > 0
                    date1 = endDate;
                end
                    
                P0 = survProbCurve.DefaultProbFromDate(date0);
                endDiscount = discountCurve.DFFromDate(paymentDate);
                
                while DateDiff(date0,endDate) < 0
                    if riskyFixedBond.paysAtDefaultTime
                        B =  discountCurve.DFFromDate(date1);
                    else
                        B = endDiscount;
                    end
                    
                    P1 = survProbCurve.DefaultProbFromDate(date1);
                    dP = P1 - P0;
                    
                    % at default cashflow
                    claim = riskyFixedBond.nominal * (recoveryRate);

                    if riskyFixedBond.paysAtDefaultTime
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    else
                        defaultLegNPV = defaultLegNPV + dP * claim * B;
                    end
                    
                    % setup for next time around the loop
                    P0 = P1;
                    date0 = date1;
                    date1 = date0.AddDate(stepSize,'day');
                    if DateDiff(date1,endDate) > 0
                        date1 = endDate;
                    end
                    
                end
                
%                 claim = riskyFixedBond.nominal * (recoveryRate);
%                 defaultLegNPV = defaultLegNPV + P * claim * discountCurve.DFFromDate(defaultDate);
                
                %loop iteration
                startDate =  endDate;
            end

            
            out.NPV = defaultLegNPV + couponLegNPV + redemptionNPV;
            out.fairRate = (riskyFixedBond.nominal * 1.0 - defaultLegNPV - redemptionNPV)*riskyFixedBond.rate/couponLegNPV;
            
        end
        
    end
    
end



In [ ]:
function out = RebonatoCorr3(rho_inf,rho_decay,alpha,tnr)
           
           corr = zeros(length(tnr),length(tnr));
           for i=1:length(tnr)
               for j=1:length(tnr)
                   beta = rho_decay*exp(-alpha*min(tnr(i),tnr(j)));
                   corr(i,j) = rho_inf + (1 - rho_inf)*exp(-beta*abs(tnr(i)-tnr(j)));
               end
           end
           out =  corr;
end



In [ ]:
function out = RebonatoCorr2(rho_inf,rho_decay,tnr)
           
           corr = zeros(length(tnr),length(tnr));
           for i=1:length(tnr)
               for j=1:length(tnr)
                    corr(i,j) = rho_inf + (1 - rho_inf)*exp(-rho_decay*abs(tnr(i)-tnr(j)));
               end
           end
           out =  corr;
end



In [ ]:
classdef RangeAccrual < Security
    %RangeAccrual is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        nominal
        coupon
        couponFreq
        tenorL
        tenorS
        lower
        upper
        schedule
    end
    
    methods
        % constructor
        function rangeAccrual = RangeAccrual(params)
                rangeAccrual.payCcy = params.payCcy;
                rangeAccrual.startDate = params.startDate;
                rangeAccrual.maturity = params.maturity;
                
                ragneAccrual.maturityDate = params.startDate.AddDate(params.maturity,'year');
                rangeAccrual.nominal = params.params('nominal');
                rangeAccrual.coupon = params.params('coupon');
                rangeAccrual.couponFreq= params.params('couponFreq');
                rangeAccrual.tenorL = params.params('tenorL');
                rangeAccrual.tenorS = params.params('tenorS');
                rangeAccrual.lower = params.params('lower');
                rangeAccrual.upper = params.params('upper');
                rangeAccrual.schedule = rangeAccrual.ScheduleGenerate(rangeAccrual.startDate,ragneAccrual.maturityDate,rangeAccrual.couponFreq);
                 
        end
        
        function out = ScheduleGenerate(rangeAccrual,initialDate,maturityDate,freq)
            schedule = ScheduleGenerate(initialDate,maturityDate,freq);
            out.startDates= schedule.startDates;
            out.resetDates = schedule.resetDates;
            out.endDates = schedule.endDates;
            out.payDates = schedule.payDates;
            out.dayCountFraction = schedule.dayCountFraction;
            out.numOfPeriod = schedule.numOfPeriod;
        end
        
        function out = NumMethodTimeStepsGenerate(rangeAccrual,valueDate,schedule,mcOneTimeStep)
            periodCount = schedule.numOfPeriod;
            numOfAlivePeriod = 0;
            for i=1:periodCount
                timeSteps = [];
                if DateDiff(valueDate,schedule.payDates(i)) >= 0
                    out.timeStepsPerPeriod(i) = timeSteps;
                    continue;
                end
                
                numOfAlivePeriod = numOfAlivePeriod + 1;
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    startDate = valueDate;
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    timeSteps = [];
                    numOfTimeSteps = 0;
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endTime];
                        numOfTimeSteps = numOfTimeSteps + 1;
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    timeSteps = sort(timeSchedule,'ascend');
                    out.timeStepsPerPeriod(i) = timeSteps;
                    out.numOfTimeSteps(i) = numOfTimeSteps;
                else
                    startDate = schedule.startDates(i);
                    endDate = schedule.endDates(i);
                    % we exclude end date from the 
                    endDate = endDate.AddDate(-1,'day');
                    timeSteps = [];
                    numOfTimeSteps = 0;
                    while 1
                        endTime = endDate.DateDiff(valueDate);
                        timeSteps = [timeSteps; endDate];
                        numOfTimeSteps = numOfTimeSteps + 1;
                        endDate = endDate.AddDate(-1*mcOneTimeStep,'day');
                        if endDate.DateDiff(startDate) < 0 break;end
                    end
                    out.timeStepsPerPeriod(i) = timeSteps;
                    out.numOfTimeSteps(i) = numOfTimeSteps;
                end
            end
            out.periodCount = periodCount;
            out.numOfAlivePeriod = numOfAlivePeriod;
        end
        
        function out = computePrice(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = zeros(periodCount,1);
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                else
                    observeDate = schedule.startDates(i);
                end
                
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    to = DateDiff(observeDate, valueDate)/365.0;
                    tp = DateDiff(schedule.payDates(i), valueDate)/365.0;
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    %rangeValue = model.CMSSpreadDigitalRange(to,tp,tenor1,tenor2,tau,lower,upper);
                    rangeValue = model.CMSSpreadDigitalRangeDate(to,tp,tenor1,tenor2,tau,lower,upper);
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                rangeProb(i) = rangeProb(i)/observeDays;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
        
        function out = computePriceMCForward(rangeAccrual,valueDate,model)
            schedule = rangeAccrual.schedule;
            
            % numMethodTimeSteps Generate
            mcOneTimeStep = 7;
            out = rangeAccrual.NumMethodTimeStepsGenerate(valueDate,schedule,mcOneTimeStep);
            
            %
            periodCount = schedule.numOfPeriod;
            rangeProb = zeros(periodCount,1);
            discountFactor = zeros(periodCount,1);
            daycountFraction = zeros(periodCount,1);
            cashflow = zeros(periodCount,1);
            cashflowNpv = zeros(periodCount,1);
            rangeDays = zeros(periodCount,1);
            npv = 0;
            for i=1:periodCount
                if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end;
                
                observeDate = schedule.startDates(i);
                if DateDiff(valueDate,schedule.startDates(i)) >= 0
                    observeDate = valueDate;
                    observeDate = AddDate(observeDate,1,'day');
                    
                else
                    observeDate = schedule.startDates(i);
                end
                
                    
                observeDays = DateDiff(schedule.endDates(i),schedule.startDates(i));
                days = DateDiff(schedule.endDates(i), observeDate);
                discountFactor(i) = model.DF( DateDiff(schedule.payDates(i),valueDate)/365.0);
                daycountFraction(i) = schedule.dayCountFraction(i);
                for j=1:days
                    to = DateDiff(observeDate, valueDate)/365.0;
                    tp = DateDiff(schedule.payDates(i), valueDate)/365.0;
                    tenor1 = rangeAccrual.tenorL;
                    tenor2 = rangeAccrual.tenorS;
                    tau =0.25;
                    lower = rangeAccrual.lower;
                    upper = rangeAccrual.upper;
                    %rangeValue = model.CMSSpreadDigitalRange(to,tp,tenor1,tenor2,tau,lower,upper);
                    rangeValue = model.CMSSpreadDigitalRangeDate(to,tp,tenor1,tenor2,tau,lower,upper);
                    rangeProb(i) = rangeProb(i) + rangeValue;
                    observeDate = AddDate(observeDate,1,'day');
                end
                cashflow(i) = rangeAccrual.nominal * rangeAccrual.coupon * (rangeProb(i) + rangeDays(i))/observeDays * daycountFraction(i);
                cashflowNpv(i) = cashflow(i) * discountFactor(i);
                npv = npv + cashflowNpv(i);
                rangeProb(i) = rangeProb(i)/observeDays;
                
                if i == periodCount
                    npv = npv + rangeAccrual.nominal * discountFactor(i);
                end
            end
            out.cashflow= cashflow;
            out.cashflowNpv = cashflowNpv;
            out.discountFactor = discountFactor;
            out.daycountFraction = daycountFraction;
            out.rangeProb = rangeProb;
            out.npv = npv;
        end
       
        
    end
    
end



In [ ]:
function value = r8poly_value ( n, a, x )

%*****************************************************************************80
%
%% R8POLY_VALUE evaluates an R8 polynomial.
%
%  Discussion:
%
%    For sanity's sake, the value of N indicates the NUMBER of 
%    coefficients, or more precisely, the ORDER of the polynomial,
%    rather than the DEGREE of the polynomial.  The two quantities
%    differ by 1, but cause a great deal of confusion.
%
%    Given N and A, the form of the polynomial is:
%
%      p(x) = a(1) + a(2) * x + ... + a(n-1) * x^(n-2) + a(n) * x^(n-1)
%
%  Licensing:
%
%    This code is distributed under the GNU LGPL license.
%
%  Modified:
%
%    09 April 2005
%
%  Author:
%
%    John Burkardt
%
%  Parameters:
%
%    Input, integer N, the order of the polynomial.
%
%    Input, real A(1:N), the coefficients of the polynomial.
%    A(1) is the constant term.
%
%    Input, real X, the point at which the polynomial is to be evaluated.
%
%    Output, real VALUE, the value of the polynomial at X.
%
  value = 0.0;
  for i = n : -1 : 1
    value = value * x + a(i);
  end

  return
end


In [ ]:
function value = r8_normal_01_cdf_inverse ( p )

%*****************************************************************************80
%
%% R8_NORMAL_01_CDF_INVERSE inverts the standard normal CDF.
%
%  Discussion:
%
%    The result is accurate to about 1 part in 10**16.
%
%  Licensing:
%
%    This code is distributed under the GNU LGPL license.
%
%  Modified:
%
%    27 December 2004
%
%  Author:
%
%    Original FORTRAN77 version by Michael Wichura.
%    MATLAB version by John Burkardt.
%
%  Reference:
%
%    Michael Wichura,
%    The Percentage Points of the Normal Distribution,
%    Algorithm AS 241,
%    Applied Statistics,
%    Volume 37, Number 3, pages 477-484, 1988.
%
%  Parameters:
%
%    Input, real P, the value of the cumulative probability 
%    densitity function.  0 < P < 1.  If P is outside this range, an
%    "infinite" value is returned.
%
%    Output, real VALUE, the normal deviate value with the 
%    property that the probability of a standard normal deviate being 
%    less than or equal to the value is P.
%
  a = [ 3.3871328727963666080,      1.3314166789178437745e+2, ...
        1.9715909503065514427e+3,   1.3731693765509461125e+4, ...
        4.5921953931549871457e+4,   6.7265770927008700853e+4, ...
        3.3430575583588128105e+4,   2.5090809287301226727e+3 ];
  b = [ 1.0,                        4.2313330701600911252e+1, ...
        6.8718700749205790830e+2,   5.3941960214247511077e+3, ...
        2.1213794301586595867e+4,   3.9307895800092710610e+4, ...
        2.8729085735721942674e+4,   5.2264952788528545610e+3 ];
  c = [ 1.42343711074968357734,     4.63033784615654529590, ...
        5.76949722146069140550,     3.64784832476320460504, ...
        1.27045825245236838258,     2.41780725177450611770e-1, ...
        2.27238449892691845833e-2,  7.74545014278341407640e-4 ];
  const1 = 0.180625;
  const2 = 1.6;
  d = [ 1.0,                        2.05319162663775882187,    ...
        1.67638483018380384940,     6.89767334985100004550e-1, ...
        1.48103976427480074590e-1,  1.51986665636164571966e-2, ...
        5.47593808499534494600e-4,  1.05075007164441684324e-9 ];
  e = [ 6.65790464350110377720,     5.46378491116411436990,    ...
        1.78482653991729133580,     2.96560571828504891230e-1, ...
        2.65321895265761230930e-2,  1.24266094738807843860e-3, ...
        2.71155556874348757815e-5,  2.01033439929228813265e-7 ];
  f = [ 1.0,                        5.99832206555887937690e-1, ...
        1.36929880922735805310e-1,  1.48753612908506148525e-2, ...
        7.86869131145613259100e-4,  1.84631831751005468180e-5, ...
        1.42151175831644588870e-7,  2.04426310338993978564e-15 ];
  split1 = 0.425;
  split2 = 5.0;

  if ( p <= 0.0 )
    value = -r8_huge ( );
    return
  end

  if ( 1.0 <= p )
    value = r8_huge ( );
    return
  end

  q = p - 0.5;

  if ( abs ( q ) <= split1 )

    r = const1 - q * q;
    value = q * r8poly_value ( 8, a, r ) / r8poly_value ( 8, b, r );

  else

    if ( q < 0.0 )
      r = p;
    else
      r = 1.0 - p;
    end

    if ( r <= 0.0 )
      value = -1.0;
      error ( 'R8_NORMAL_CDF_INVERSE - Fatal error!' );
    end

    r = sqrt ( -log ( r ) );

    if ( r <= split2 )

      r = r - const2;
      value = r8poly_value ( 8, c, r ) / r8poly_value ( 8, d, r );

     else

       r = r - split2;
       value = r8poly_value ( 8, e, r ) / r8poly_value ( 8, f, r );

    end

    if ( q < 0.0 )
      value = -value;
    end

  end

  return
end


In [ ]:
function value = r8_huge ( )

%*****************************************************************************80
%
%% R8_HUGE returns a "huge" real number.
%
%  Discussion:
%
%    The value returned by this function is NOT required to be the
%    maximum representable R8.  This value varies from machine to machine,
%    from compiler to compiler, and may cause problems when being printed.
%    We simply want a "very large" but non-infinite number.
%
%    MATLAB provides a built-in symbolic constant "inf" that can be used
%    if a huge number is really what you want!
%
%  Licensing:
%
%    This code is distributed under the GNU LGPL license.
%
%  Modified:
%
%    27 January 2008
%
%  Author:
%
%    John Burkardt
%
%  Parameters:
%
%    Output, real VALUE, a huge number.
%
  value = 1.0E+30;

  return
end


In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
